In [ ]:
import os
import numpy as np
import pandas as pd
import scipy as sci
from scipy import special, stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
import fc_methods as fc
import helpers as hp

In [ ]:
# give the source to all datasets:
foldername_sample_fMRI = '/Users/nataliabielczyk/Desktop/variousprojects/AoN_Brainhack/PIPELINE/FunctionalConnectivity_AoNBrainhackWarsaw-master/codes/'
foldername_data_fMRI = '/Users/nataliabielczyk/Desktop/variousprojects/AoN_Brainhack/PIPELINE/FunctionalConnectivity_AoNBrainhackWarsaw-master/datasets/fMRI/'
foldername_data_MEG = '/Users/nataliabielczyk/Desktop/variousprojects/AoN_Brainhack/PIPELINE/FunctionalConnectivity_AoNBrainhackWarsaw-master/datasets/MEG/'
foldername_data_google_trends = '/Users/nataliabielczyk/Desktop/variousprojects/AoN_Brainhack/PIPELINE/FunctionalConnectivity_AoNBrainhackWarsaw-master/datasets/google_trends/'
foldername_data_stock_exchange = '/Users/nataliabielczyk/Desktop/variousprojects/AoN_Brainhack/PIPELINE/FunctionalConnectivity_AoNBrainhackWarsaw-master/datasets/stock_exchange/'
foldername_data_weather = '/Users/nataliabielczyk/Desktop/variousprojects/AoN_Brainhack/PIPELINE/FunctionalConnectivity_AoNBrainhackWarsaw-master/datasets/weather/'

In [ ]:
# give the output folder addresses:
output_fMRI = '/Users/nataliabielczyk/Desktop/variousprojects/AoN_Brainhack/PIPELINE/FunctionalConnectivity_AoNBrainhackWarsaw-master/results/fMRI/'
output_MEG = '/Users/nataliabielczyk/Desktop/variousprojects/AoN_Brainhack/PIPELINE/FunctionalConnectivity_AoNBrainhackWarsaw-master/results/MEG/'
output_google_trends = '/Users/nataliabielczyk/Desktop/variousprojects/AoN_Brainhack/PIPELINE/FunctionalConnectivity_AoNBrainhackWarsaw-master/results/google_trends/'
output_stock_exchange = '/Users/nataliabielczyk/Desktop/variousprojects/AoN_Brainhack/PIPELINE/FunctionalConnectivity_AoNBrainhackWarsaw-master/results/stock_exchange/'
output_weather = '/Users/nataliabielczyk/Desktop/variousprojects/AoN_Brainhack/PIPELINE/FunctionalConnectivity_AoNBrainhackWarsaw-master/results/weather/'
output_Berkson = '/Users/nataliabielczyk/Desktop/variousprojects/AoN_Brainhack/PIPELINE/FunctionalConnectivity_AoNBrainhackWarsaw-master/results/Berkson/'
output_synthetic = '/Users/nataliabielczyk/Desktop/variousprojects/AoN_Brainhack/PIPELINE/FunctionalConnectivity_AoNBrainhackWarsaw-master/results/synthetic/'

## Example comparison of correlation methods on random data

First, let us compare Pearson and partial correlation on randomly generated data sampled from a uniform distribution.

We will start by creating the data matrix with 50 features and 300 "timepoints".

In [ ]:
np.random.seed(1234)

In [ ]:
rand_matrix = np.random.rand(300,50)

Now, let's calculate the correlation and p-values matrices using both methods.

In [ ]:
pearson_random, pearson_random_p = fc.pearson_corr(rand_matrix)

In [ ]:
partial_random, partial_random_p = fc.partial_corr(rand_matrix)

In [ ]:
partial_partial_random, partial_partial_random_p = fc.partial_partial_corr(rand_matrix,1.0)

In [ ]:
# compare two implementatiosn of partial correlation on random data:
fig = plt.figure(figsize=(5,4.5))
sns.heatmap(partial_random, vmin=-0.15, vmax=0.15, cmap='coolwarm');
fig = plt.figure(figsize=(5,4.5))
sns.heatmap(partial_partial_random, vmin=-0.15, vmax=0.15, cmap='coolwarm');

We will visualise them side by side, so the differences will be easier to spot.

In [ ]:
# plot Pearson versus partial correlation, together with the associated statistics p:
def plot_corr(pearson, pearson_p, partial_partial, partial_partial_p, partial, partial_p):
    fig = plt.figure(figsize=(25,11))
    #fig.suptitle('sample fMRI subject, correlations', y=1.05, fontsize=20);
    
    ax  = plt.subplot2grid((3,6),(0,0), colspan=2, rowspan=2)
    ax1 = plt.subplot2grid((3,6),(0,2), colspan=2, rowspan=2)
    ax2 = plt.subplot2grid((3,6),(0,4), colspan=2, rowspan=2)
    
    sns.heatmap(pearson, vmin=-0.5, vmax=0.5, cmap='coolwarm', ax=ax);
    ax.set_xlabel('Pearson correlation', fontsize=16);#, y=1.10);
    
    sns.heatmap(partial_partial, vmin=-0.5, vmax=0.5, cmap='coolwarm', ax=ax1);
    ax1.set_xlabel('Partial partial correlation', fontsize=16);#, y=1.10);
    
    sns.heatmap(partial, vmin=-0.5, vmax=0.5, cmap='coolwarm', ax=ax2);
    ax2.set_xlabel('Partial correlation', fontsize=16);#, y=1.10);
    

    ax3 = plt.subplot2grid((3,6),(2,0))
    ax4 = plt.subplot2grid((3,6),(2,1))
    
    if type(pearson) == type(pd.DataFrame()):
        sns.distplot(pearson.values.flatten()[np.where(pearson.values.flatten()<1)], kde=False, ax=ax3);
        sns.distplot(pearson_p.values.flatten(), kde=False, ax=ax4);
    else:
        sns.distplot(pearson.flatten()[np.where(pearson.flatten()<1)], kde=False, ax=ax3);
        sns.distplot(pearson_p.flatten(), kde=False, ax=ax4);
    ax3.set_title('Distribution of Pearson correlation values', y=1.10);
    ax4.set_title('P-values distribution', y=1.10);

    ax5 = plt.subplot2grid((3,6),(2,2))
    ax6 = plt.subplot2grid((3,6),(2,3))
    if type(partial_partial) == type(pd.DataFrame()):
        sns.distplot(partial_partial.values.flatten()[np.where(partial_partial.values.flatten()<1)], kde=False, ax=ax5);
        sns.distplot(partial_partial_p.values.flatten(), kde=False, ax=ax6);
    else:
        sns.distplot(partial_partial.flatten()[np.where(partial_partial.flatten()<1)], kde=False, ax=ax5);
        sns.distplot(partial_partial_p.flatten(), kde=False, ax=ax6);
    ax5.set_title('Distribution of partial partial correlation values', y=1.10);
    ax6.set_title('P-values distribution', y=1.10);

    ax7 = plt.subplot2grid((3,6),(2,4))
    ax8 = plt.subplot2grid((3,6),(2,5))
    if type(partial) == type(pd.DataFrame()):
        sns.distplot(partial.values.flatten()[np.where(partial.values.flatten()<1)], kde=False, ax=ax7);
        sns.distplot(partial_p.values.flatten(), kde=False, ax=ax8);
    else:
        sns.distplot(partial.flatten()[np.where(partial.flatten()<1)], kde=False, ax=ax7);
        sns.distplot(partial_p.flatten(), kde=False, ax=ax8);
    ax7.set_title('Distribution of partial correlation values', y=1.10);
    ax8.set_title('P-values distribution', y=1.10);
    
    for ax in fig.axes:
        plt.sca(ax)
        ax.xaxis.tick_top()
        plt.xticks(rotation=90)       
                  
    plt.tight_layout()
    
    return fig

In [ ]:
def plot_all(pearson, partial_partial, partial, mi, inv_ed, inv_md, max_pearson, max_partial_partial, max_partial, max_mi, max_inv_ed, max_inv_md):
    
    fig = plt.figure(figsize=(21,12))
    #fig.suptitle('sample fMRI subject', y=1.25, fontsize=20);
    
    ax  = plt.subplot2grid((2,3),(0,0), colspan=1, rowspan=1)
    ax1 = plt.subplot2grid((2,3),(0,1), colspan=1, rowspan=1)
    ax2 = plt.subplot2grid((2,3),(0,2), colspan=1, rowspan=1)
    ax3 = plt.subplot2grid((2,3),(1,0), colspan=1, rowspan=1)
    ax4 = plt.subplot2grid((2,3),(1,1), colspan=1, rowspan=1)
    ax5 = plt.subplot2grid((2,3),(1,2), colspan=1, rowspan=1)
    
    sns.heatmap(pearson, vmin=-abs(max_pearson), vmax=abs(max_pearson), cmap='coolwarm', ax=ax);
    ax.set_xlabel('Pearson correlation', fontsize=16);#, y=1.10);
    
    sns.heatmap(partial_partial, vmin=-abs(max_partial_partial), vmax=abs(max_partial_partial), cmap='coolwarm', ax=ax1);
    ax1.set_xlabel('Partial partial correlation', fontsize=16);#, y=1.10);
    
    sns.heatmap(partial, vmin=-abs(max_partial), vmax=abs(max_partial), cmap='coolwarm', ax=ax2);
    ax2.set_xlabel('Partial correlation', fontsize=16);#, y=1.10);
    
    sns.heatmap(mi, vmin=-abs(max_mi), vmax=abs(max_mi), cmap='coolwarm', ax=ax3);
    ax3.set_xlabel('Mutual information', fontsize=16);#, y=1.10);
    
    sns.heatmap(inv_ed, vmin=-abs(max_inv_ed), vmax=abs(max_inv_ed), cmap='coolwarm', ax=ax4);
    ax4.set_xlabel('Inverse Euclidean distance', fontsize=16);#, y=1.10);
    
    sns.heatmap(inv_md, vmin=-abs(max_inv_md), vmax=abs(max_inv_md), cmap='coolwarm', ax=ax5);
    ax5.set_xlabel('Inverse Manhattan distance', fontsize=16);#, y=1.10);
    
    for ax in fig.axes:
        plt.sca(ax)
        ax.xaxis.tick_top()
        plt.xticks(rotation=90)
        
            
    plt.tight_layout()
    
    return fig

As one could expect, when dealing with random data, there is practically no difference between Pearson and partial correlation.  
Each variable is fairly independent from others, so partial correlation gives the same results (no additional correlations that we have to control for).

## Example comparison of correlation methods on generated correlated data

Next, we will see how these methods will work with correlated data.  
We will generate some synthetic data which will follow a predefined correlation structure.  

In [ ]:
network_size         = 10
network_density      = 0.25
conn_weight          = 0.15

noise_level          = 0.1   
T                    = 1000  
dt                   = 0.01  
    
true_fc = fc.generate_fc(network_size, network_density, conn_weight) 
data_synthetic = fc.simulate_data(true_fc, noise_level, T, dt)
n = data_synthetic.shape[0]
m = data_synthetic.shape[1]

In [ ]:
# print true connectivity:
x = np.arange(network_size)+0.5
labels = np.ndarray.tolist(np.arange(network_size)+1)

fig = plt.figure(figsize=(7,6))
sns.heatmap(true_fc, vmin=0, vmax=0.1, cmap='coolwarm');
ax = plt.axes()
ax.set_title('True underlying functional connectivity')
plt.xticks(x, labels)
plt.yticks(x, labels)
plt.tight_layout()
plt.show()

folder_output = output_synthetic
fname = folder_output + 'true_fc'

dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

hp.savef(fig, fname, dpis, formats)

In [ ]:
# Pearson correlation:
pearson_synthetic, pearson_synthetic_p = (pd.DataFrame(fc.pearson_corr(data_synthetic)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.pearson_corr(data_synthetic)[1], 
                                          index=labels, columns=labels))
max_pearson_synthetic = np.max(fc.pearson_corr(data_synthetic)[0][np.triu_indices(m,1)])

# Partial partial correlation:
alpha = 0.5
partial_partial_synthetic, partial_partial_synthetic_p = (pd.DataFrame(fc.partial_partial_corr(data_synthetic,alpha)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.partial_partial_corr(data_synthetic,alpha)[1], 
                                          index=labels, columns=labels))
max_partial_partial_synthetic = np.max(fc.partial_partial_corr(data_synthetic,alpha)[0][np.triu_indices(m,1)])

# Partial correlation:
alpha = 1.0
partial_synthetic, partial_synthetic_p = (pd.DataFrame(fc.partial_partial_corr(data_synthetic,alpha)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.partial_partial_corr(data_synthetic,alpha)[1], 
                                          index=labels, columns=labels))
max_partial_synthetic = np.max(fc.partial_partial_corr(data_synthetic,alpha)[0][np.triu_indices(m,1)])

# Mutual Information:
# take number of bins as square root of the data length:
nbins = int(np.floor(np.sqrt(n)))
mi_synthetic = np.zeros((m,m))
for i in range(m):
    mi_synthetic[i, i] = 10
    for j in range(i+1, m):
        mi_synthetic[i,j] = fc.calc_MI(data_synthetic[:,i], data_synthetic[:,j], nbins)
        mi_synthetic[j,i] = mi_synthetic[i,j]

max_mi_synthetic = np.max(mi_synthetic[np.triu_indices(m,1)])
mi_synthetic = pd.DataFrame(mi_synthetic, index=labels, columns=labels)

# inverse of Euclidean distance:
inv_ed_synthetic = np.zeros((m,m))
for i in range(m):
    inv_ed_synthetic[i, i] = 1.0
    for j in range(i+1, m):
        inv_ed_synthetic[i,j] = 1.0/fc.euclidean_distance(data_synthetic[:,i], data_synthetic[:,j])
        inv_ed_synthetic[j,i] = inv_ed_synthetic[i,j]
        
max_inv_ed_synthetic = np.max(inv_ed_synthetic[np.triu_indices(m,1)])
inv_ed_synthetic = pd.DataFrame(inv_ed_synthetic, index=labels, columns=labels)

# inverse of Manhattan distance:
inv_md_synthetic = np.zeros((m,m))
for i in range(m):
    inv_md_synthetic[i, i] = 1.0
    for j in range(i+1, m):
        inv_md_synthetic[i,j] = 1.0/fc.manhattan_distance(data_synthetic[:,i], data_synthetic[:,j])
        inv_md_synthetic[j,i] = inv_md_synthetic[i,j]
        
max_inv_md_synthetic = np.max(inv_md_synthetic[np.triu_indices(m,1)])
inv_md_synthetic = pd.DataFrame(inv_md_synthetic, index=labels, columns=labels)

corr_synthetic = plot_corr(pearson_synthetic, pearson_synthetic_p, partial_partial_synthetic, partial_partial_synthetic_p, partial_synthetic, partial_synthetic_p)

all_synthetic = plot_all(pearson_synthetic, partial_partial_synthetic, partial_synthetic, mi_synthetic, inv_ed_synthetic, inv_md_synthetic, max_pearson_synthetic, max_partial_partial_synthetic, max_partial_synthetic, max_mi_synthetic, max_inv_ed_synthetic, max_inv_md_synthetic)

folder_output = output_synthetic
fname = folder_output + 'all_methods_synthetic'

dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

hp.savef(all_synthetic, fname, dpis, formats)

# plot and save all the clustermaps:
folder_output = output_synthetic
dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

# Pearson correlation:
fig1 = plt.figure(figsize=(12,11))
s1 = sns.clustermap(pearson_synthetic, vmin=-abs(max_pearson_synthetic), vmax=abs(max_pearson_synthetic), cmap='coolwarm');
plt.setp(s1.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s1.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Pearson correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_pearson_synthetic_' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Partial partial correlation:
fig2 = plt.figure(figsize=(12,11))
s2 = sns.clustermap(partial_partial_synthetic, vmin=-abs(max_partial_partial_synthetic), vmax=abs(max_partial_partial_synthetic), cmap='coolwarm');
plt.setp(s2.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s2.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Partial partial correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_partial_partial_synthetic_' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Partial correlation: 
fig3 = plt.figure(figsize=(12,11))
s3 = sns.clustermap(partial_synthetic, vmin=-abs(max_partial_synthetic), vmax=abs(max_partial_synthetic), cmap='coolwarm');
plt.setp(s3.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s3.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Partial correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_partial_synthetic_' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Mutual information: 
fig4 = plt.figure(figsize=(12,11))
s4 = sns.clustermap(mi_synthetic, vmin=-abs(max_mi_synthetic), vmax=abs(max_mi_synthetic), cmap='coolwarm');
plt.setp(s4.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s4.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Mutual information', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_mi_synthetic_' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

# Euclidean distance: 
fig5 = plt.figure(figsize=(12,11))
s5 = sns.clustermap(inv_ed_synthetic, vmin=-abs(max_inv_ed_synthetic), vmax=abs(max_inv_ed_synthetic), cmap='coolwarm');
plt.setp(s5.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s5.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Inverse Euclidean distance', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_inv_ed_synthetic_' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

# Manhattan distance: 
fig6 = plt.figure(figsize=(12,11))
s6 = sns.clustermap(inv_md_synthetic, vmin=-abs(max_inv_md_synthetic), vmax=abs(max_inv_md_synthetic), cmap='coolwarm');
plt.setp(s6.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s6.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Inverse Manhattan distance', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_inv_md_synthetic_' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

        
        
# add detrending:
data_synthetic_detrended = fc.detrend(data_synthetic)

# Pearson correlation:
pearson_synthetic_detrended, pearson_synthetic_p_detrended = (pd.DataFrame(fc.pearson_corr(data_synthetic_detrended)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.pearson_corr(data_synthetic_detrended)[1], 
                                          index=labels, columns=labels))
max_pearson_synthetic_detrended = np.max(fc.pearson_corr(data_synthetic_detrended)[0][np.triu_indices(m,1)])

# Partial partial correlation:
alpha = 0.5
partial_partial_synthetic_detrended, partial_partial_synthetic_p_detrended = (pd.DataFrame(fc.partial_partial_corr(data_synthetic_detrended,alpha)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.partial_partial_corr(data_synthetic_detrended,alpha)[1], 
                                          index=labels, columns=labels))
max_partial_partial_synthetic_detrended = np.max(fc.partial_partial_corr(data_synthetic_detrended,alpha)[0][np.triu_indices(m,1)])

# Partial correlation:
alpha = 1.0
partial_synthetic_detrended, partial_synthetic_p_detrended = (pd.DataFrame(fc.partial_partial_corr(data_synthetic_detrended,alpha)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.partial_partial_corr(data_synthetic_detrended,alpha)[1], 
                                          index=labels, columns=labels))
max_partial_synthetic_detrended = np.max(fc.partial_partial_corr(data_synthetic_detrended,alpha)[0][np.triu_indices(m,1)])

# Mutual Information:
# take number of bins as square root of the data length:
nbins = int(np.floor(np.sqrt(n)))
mi_synthetic_detrended = np.zeros((m,m))
for i in range(m):
    mi_synthetic_detrended[i, i] = 10
    for j in range(i+1, m):
        mi_synthetic_detrended[i,j] = fc.calc_MI(data_synthetic_detrended[:,i], data_synthetic_detrended[:,j], nbins)
        mi_synthetic_detrended[j,i] = mi_synthetic_detrended[i,j]

max_mi_synthetic_detrended = np.max(mi_synthetic_detrended[np.triu_indices(m,1)])
mi_synthetic_detrended = pd.DataFrame(mi_synthetic_detrended, index=labels, columns=labels)

# inverse of Euclidean distance:
inv_ed_synthetic_detrended = np.zeros((m,m))
for i in range(m):
    inv_ed_synthetic_detrended[i, i] = 1.0
    for j in range(i+1, m):
        inv_ed_synthetic_detrended[i,j] = 1.0/fc.euclidean_distance(data_synthetic_detrended[:,i], data_synthetic_detrended[:,j])
        inv_ed_synthetic_detrended[j,i] = inv_ed_synthetic_detrended[i,j]
        
max_inv_ed_synthetic_detrended = np.max(inv_ed_synthetic_detrended[np.triu_indices(m,1)])
inv_ed_synthetic_detrended = pd.DataFrame(inv_ed_synthetic_detrended, index=labels, columns=labels)

# inverse of Manhattan distance:
inv_md_synthetic_detrended = np.zeros((m,m))
for i in range(m):
    inv_md_synthetic_detrended[i, i] = 1.0
    for j in range(i+1, m):
        inv_md_synthetic_detrended[i,j] = 1.0/fc.manhattan_distance(data_synthetic_detrended[:,i], data_synthetic_detrended[:,j])
        inv_md_synthetic_detrended[j,i] = inv_md_synthetic_detrended[i,j]
        
max_inv_md_synthetic_detrended = np.max(inv_md_synthetic_detrended[np.triu_indices(m,1)])
inv_md_synthetic_detrended = pd.DataFrame(inv_md_synthetic_detrended, index=labels, columns=labels)

corr_synthetic_detrended = plot_corr(pearson_synthetic_detrended, pearson_synthetic_p_detrended, partial_partial_synthetic_detrended, partial_partial_synthetic_p_detrended, partial_synthetic_detrended, partial_synthetic_p_detrended)

all_synthetic_detrended = plot_all(pearson_synthetic_detrended, partial_partial_synthetic_detrended, partial_synthetic_detrended, mi_synthetic_detrended, inv_ed_synthetic_detrended, inv_md_synthetic_detrended, max_pearson_synthetic_detrended, max_partial_partial_synthetic_detrended, max_partial_synthetic_detrended, max_mi_synthetic_detrended, max_inv_ed_synthetic_detrended, max_inv_md_synthetic_detrended)

folder_output = output_synthetic
fname = folder_output + 'all_methods_synthetic_detrended'

dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

hp.savef(all_synthetic_detrended, fname, dpis, formats)

# plot and save all the clustermaps:
folder_output = output_synthetic
dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

# Pearson correlation:
fig1 = plt.figure(figsize=(12,11))
s1 = sns.clustermap(pearson_synthetic_detrended, vmin=-abs(max_pearson_synthetic_detrended), vmax=abs(max_pearson_synthetic_detrended), cmap='coolwarm');
plt.setp(s1.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s1.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Pearson correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_pearson_synthetic_detrended_' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Partial partial correlation:
fig2 = plt.figure(figsize=(12,11))
s2 = sns.clustermap(partial_partial_synthetic_detrended, vmin=-abs(max_partial_partial_synthetic_detrended), vmax=abs(max_partial_partial_synthetic_detrended), cmap='coolwarm');
plt.setp(s2.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s2.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Partial partial correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_partial_partial_synthetic_detrended_' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Partial correlation: 
fig3 = plt.figure(figsize=(12,11))
s3 = sns.clustermap(partial_synthetic_detrended, vmin=-abs(max_partial_synthetic_detrended), vmax=abs(max_partial_synthetic_detrended), cmap='coolwarm');
plt.setp(s3.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s3.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Partial correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_partial_synthetic_detrended_' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Mutual information: 
fig4 = plt.figure(figsize=(12,11))
s4 = sns.clustermap(mi_synthetic_detrended, vmin=-abs(max_mi_synthetic_detrended), vmax=abs(max_mi_synthetic_detrended), cmap='coolwarm');
plt.setp(s4.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s4.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Mutual information', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_mi_synthetic_detrended' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

# Euclidean distance: 
fig5 = plt.figure(figsize=(12,11))
s5 = sns.clustermap(inv_ed_synthetic_detrended, vmin=-abs(max_inv_ed_synthetic_detrended), vmax=abs(max_inv_ed_synthetic_detrended), cmap='coolwarm');
plt.setp(s5.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s5.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Inverse Euclidean distance', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_inv_ed_synthetic_detrended' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

# Manhattan distance: 
fig6 = plt.figure(figsize=(12,11))
s6 = sns.clustermap(inv_md_synthetic_detrended, vmin=-abs(max_inv_md_synthetic_detrended), vmax=abs(max_inv_md_synthetic_detrended), cmap='coolwarm');
plt.setp(s6.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s6.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Inverse Manhattan distance', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_inv_md_synthetic_detrended' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

# plot correlation between the 12 methods:
nl = len(np.triu_indices(m,1)[0])

pearson_synthetic_vectorized = np.ndarray.reshape(np.asarray(pearson_synthetic)[np.triu_indices(m,1)], nl, 1)
partial_partial_synthetic_vectorized = np.ndarray.reshape(np.asarray(partial_partial_synthetic)[np.triu_indices(m,1)], nl, 1)
partial_synthetic_vectorized = np.ndarray.reshape(np.asarray(partial_synthetic)[np.triu_indices(m,1)], nl, 1)
mi_synthetic_vectorized = np.ndarray.reshape(np.asarray(mi_synthetic)[np.triu_indices(m,1)], nl, 1)
inv_ed_synthetic_vectorized = np.ndarray.reshape(np.asarray(inv_ed_synthetic)[np.triu_indices(m,1)], nl, 1)
inv_md_synthetic_vectorized = np.ndarray.reshape(np.asarray(inv_md_synthetic)[np.triu_indices(m,1)], nl, 1)

pearson_synthetic_detrended_vectorized = np.ndarray.reshape(np.asarray(pearson_synthetic_detrended)[np.triu_indices(m,1)], nl, 1)
partial_partial_synthetic_detrended_vectorized = np.ndarray.reshape(np.asarray(partial_partial_synthetic_detrended)[np.triu_indices(m,1)], nl, 1)
partial_synthetic_detrended_vectorized = np.ndarray.reshape(np.asarray(partial_synthetic_detrended)[np.triu_indices(m,1)], nl, 1)
mi_synthetic_detrended_vectorized = np.ndarray.reshape(np.asarray(mi_synthetic_detrended)[np.triu_indices(m,1)], nl, 1)
inv_ed_synthetic_detrended_vectorized = np.ndarray.reshape(np.asarray(inv_ed_synthetic_detrended)[np.triu_indices(m,1)], nl, 1)
inv_md_synthetic_detrended_vectorized = np.ndarray.reshape(np.asarray(inv_md_synthetic_detrended)[np.triu_indices(m,1)], nl, 1)

results_allmethods = np.concatenate((pearson_synthetic_vectorized, partial_partial_synthetic_vectorized, partial_synthetic_vectorized, \
                                     mi_synthetic_vectorized, inv_ed_synthetic_vectorized, inv_md_synthetic_vectorized, \
                                     pearson_synthetic_detrended_vectorized, partial_partial_synthetic_detrended_vectorized, partial_synthetic_detrended_vectorized, \
                                     mi_synthetic_detrended_vectorized, inv_ed_synthetic_detrended_vectorized, inv_md_synthetic_detrended_vectorized),axis=1)

corr_results_allmethods, corr_results_allmethods_p = sci.stats.spearmanr(results_allmethods)
labels_allmethods = ['Pearson correlation','Partial partial correlation', 'Partial partial correlation', \
          'Mutual information', 'Inverse Euclidean distance', 'Inverse Manhattan distance', \
          'Pearson correlation, detrended data','Partial partial correlation, detrended data', 'Partial partial correlation, detrended data', \
          'Mutual information, detrended data', 'Inverse Euclidean distance, detrended data', 'Inverse Manhattan distance, detrended data']


corr_results_allmethods = pd.DataFrame(corr_results_allmethods, index=labels_allmethods, columns=labels_allmethods)
corr_results_allmethods_p = pd.DataFrame(corr_results_allmethods_p, index=labels_allmethods, columns=labels_allmethods)

fig = plt.figure(figsize=(13,10))
ax0 = plt.subplot2grid((1,2),(0,0), colspan=1, rowspan=1)
ax1 = plt.subplot2grid((1,2),(0,1), colspan=1, rowspan=1)
sns.heatmap(corr_results_allmethods, vmin=-1.0, vmax=1.0, cmap='coolwarm', ax=ax0);
ax0.set_title('Spearman r between the methods', fontsize=16,y=1.70);
sns.heatmap(corr_results_allmethods_p, vmin=0, vmax=0.01, cmap='coolwarm', ax=ax1);
ax1.set_title('Associated p values', fontsize=16,  y=1.70);
for ax in fig.axes:
    plt.sca(ax)
    ax.axvline(x=6, color='k', linestyle='-')
    ax.axhline(y=6, color='k', linestyle='-')
    ax.xaxis.tick_top()
    plt.xticks(rotation=90)
for label in ax1.get_yticklabels():
    label.set_visible(False)
plt.tight_layout()

folder_output = output_synthetic
fname = folder_output + 'correlations_all_methods_synthetic'

dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

hp.savef(fig, fname, dpis, formats)

Now, let's calculate the correlation and p-values matrices using both methods.

We will visualise them side by side, so the differences will be easier to spot.

In [ ]:
corr_sim = plot_corr_synthetic(true_fc, pearson_sim, pearson_sim_p, partial_sim, partial_sim_p)

In [ ]:
folder_output = '../results/1_Pearson_vs_Partial/'
fname = folder_output + 'corr_sim'

dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

hp.savef(corr_sim, fname, dpis, formats)

In [ ]:
sns.clustermap(pearson_sim, vmin=-0.5, vmax=0.5, cmap='coolwarm');

## Example on fMRI resting-state data

In [ ]:
subject_label = '100307'
path = foldername_data_fMRI + 'HCP_parcellated_smoothing5_100subjects_Glasser_restingstate_day1/' + subject_label + '.csv'
label_path = foldername_data_fMRI + 'labels_Glasser.csv'
labels_all = pd.read_csv(label_path, header=None)[0].values

In [ ]:
# choose areas within:
# 1 - Primary Visual Cortex V1
# 2 - Early Visual Cortex
# 3 - Dorsal Stream Visual Cortex
# 4 - Ventral Stream Visual Cortex
# 5 - MT+ Complex and Neighboring Visual Areas
indices_visual0 = [1,2,3,4,5,6,7,11,13,16,17,18,19,20,21,22,23,50,119,121,126,127,133,135,136,137,138,140,141,142,143,146,152,153,154,155,156,157,158,159,160,163]
indices_visual = [x - 1 for x in indices_visual0]

# 6 - Somatosensory and Motor Cortex
indices_somatosensory0 = [8,9,10,11,12,36,39,40,42,47,51,52,53,54,55,56,99,100,116,117,147]
indices_somatosensory = [x - 1 for x in indices_somatosensory0]

In [ ]:
# load fMRI data:
data_fmri0 = np.asarray(pd.read_csv(path, header=None))
n = data_fmri0.shape[0]

In [ ]:
# [a] choose visual system:
data_fmri = data_fmri0[:,indices_visual]
m = data_fmri.shape[1]
labels = labels_all[indices_visual]

In [ ]:
# Pearson correlation:
pearson_fmri, pearson_fmri_p = (pd.DataFrame(fc.pearson_corr(data_fmri)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.pearson_corr(data_fmri)[1], 
                                          index=labels, columns=labels))
max_pearson_fmri = np.max(fc.pearson_corr(data_fmri)[0][np.triu_indices(m,1)])

In [ ]:
# Partial partial correlation:
alpha = 0.5
partial_partial_fmri, partial_partial_fmri_p = (pd.DataFrame(fc.partial_partial_corr(data_fmri,alpha)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.partial_partial_corr(data_fmri,alpha)[1], 
                                          index=labels, columns=labels))
max_partial_partial_fmri = np.max(fc.partial_partial_corr(data_fmri,alpha)[0][np.triu_indices(m,1)])

In [ ]:
# Partial correlation:
alpha = 1.0
partial_fmri, partial_fmri_p = (pd.DataFrame(fc.partial_partial_corr(data_fmri,alpha)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.partial_partial_corr(data_fmri,alpha)[1], 
                                          index=labels, columns=labels))
max_partial_fmri = np.max(fc.partial_partial_corr(data_fmri,alpha)[0][np.triu_indices(m,1)])

In [ ]:
# Mutual Information:
# take number of bins as square root of the data length:
nbins = int(np.floor(np.sqrt(n)))
mi_fmri = np.zeros((m,m))
for i in range(m):
    mi_fmri[i, i] = 10
    for j in range(i+1, m):
        mi_fmri[i,j] = fc.calc_MI(data_fmri[:,i], data_fmri[:,j], nbins)
        mi_fmri[j,i] = mi_fmri[i,j]

max_mi_fmri = np.max(mi_fmri[np.triu_indices(m,1)])
mi_fmri = pd.DataFrame(mi_fmri, index=labels, columns=labels)

In [ ]:
# inverse of Euclidean distance:
inv_ed_fmri = np.zeros((m,m))
for i in range(m):
    inv_ed_fmri[i, i] = 1.0
    for j in range(i+1, m):
        inv_ed_fmri[i,j] = 1.0/fc.euclidean_distance(data_fmri[:,i], data_fmri[:,j])
        inv_ed_fmri[j,i] = inv_ed_fmri[i,j]
        
max_inv_ed_fmri = np.max(inv_ed_fmri[np.triu_indices(m,1)])
inv_ed_fmri = pd.DataFrame(inv_ed_fmri, index=labels, columns=labels)

In [ ]:
# inverse of Manhattan distance:
inv_md_fmri = np.zeros((m,m))
for i in range(m):
    inv_md_fmri[i, i] = 1.0
    for j in range(i+1, m):
        inv_md_fmri[i,j] = 1.0/fc.manhattan_distance(data_fmri[:,i], data_fmri[:,j])
        inv_md_fmri[j,i] = inv_md_fmri[i,j]
        
max_inv_md_fmri = np.max(inv_md_fmri[np.triu_indices(m,1)])
inv_md_fmri = pd.DataFrame(inv_md_fmri, index=labels, columns=labels)

In [ ]:
corr_fmri = plot_corr(pearson_fmri, pearson_fmri_p, partial_partial_fmri, partial_partial_fmri_p, partial_fmri, partial_fmri_p)

In [ ]:
all_fmri = plot_all(pearson_fmri, partial_partial_fmri, partial_fmri, mi_fmri, inv_ed_fmri, inv_md_fmri, max_pearson_fmri, max_partial_partial_fmri, max_partial_fmri, max_mi_fmri, max_inv_ed_fmri, max_inv_md_fmri)

In [ ]:
folder_output = output_fMRI
fname = folder_output + 'all_methods_fmri_visual_system_subject' + subject_label

dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

hp.savef(all_fmri, fname, dpis, formats)

In [ ]:
# plot and save all the clustermaps:
folder_output = output_fMRI
dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

# Pearson correlation:
fig1 = plt.figure(figsize=(12,11))
s1 = sns.clustermap(pearson_fmri, vmin=-abs(max_pearson_fmri), vmax=abs(max_pearson_fmri), cmap='coolwarm');
plt.setp(s1.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s1.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Pearson correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_pearson_fmri_visual_system_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Partial partial correlation:
fig2 = plt.figure(figsize=(12,11))
s2 = sns.clustermap(partial_partial_fmri, vmin=-abs(max_partial_partial_fmri), vmax=abs(max_partial_partial_fmri), cmap='coolwarm');
plt.setp(s2.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s2.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Partial partial correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_partial_partial_fmri_visual_system_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Partial correlation: 
fig3 = plt.figure(figsize=(12,11))
s3 = sns.clustermap(partial_fmri, vmin=-abs(max_partial_fmri), vmax=abs(max_partial_fmri), cmap='coolwarm');
plt.setp(s3.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s3.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Partial correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_partial_fmri_visual_system_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Mutual information: 
fig4 = plt.figure(figsize=(12,11))
s4 = sns.clustermap(mi_fmri, vmin=-abs(max_mi_fmri), vmax=abs(max_mi_fmri), cmap='coolwarm');
plt.setp(s4.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s4.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Mutual information', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_mi_fmri_visual_system_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

# Euclidean distance: 
fig5 = plt.figure(figsize=(12,11))
s5 = sns.clustermap(inv_ed_fmri, vmin=-abs(max_inv_ed_fmri), vmax=abs(max_inv_ed_fmri), cmap='coolwarm');
plt.setp(s5.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s5.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Inverse Euclidean distance', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_inv_ed_fmri_visual_system_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

# Manhattan distance: 
fig6 = plt.figure(figsize=(12,11))
s6 = sns.clustermap(inv_md_fmri, vmin=-abs(max_inv_md_fmri), vmax=abs(max_inv_md_fmri), cmap='coolwarm');
plt.setp(s6.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s6.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Inverse Manhattan distance', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_inv_md_fmri_visual_system_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

In [ ]:
# add detrending:
data_fmri_detrended = fc.detrend(data_fmri)

In [ ]:
# Pearson correlation:
pearson_fmri_detrended, pearson_fmri_p_detrended = (pd.DataFrame(fc.pearson_corr(data_fmri_detrended)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.pearson_corr(data_fmri_detrended)[1], 
                                          index=labels, columns=labels))
max_pearson_fmri_detrended = np.max(fc.pearson_corr(data_fmri_detrended)[0][np.triu_indices(m,1)])

In [ ]:
# Partial partial correlation:
alpha = 0.5
partial_partial_fmri_detrended, partial_partial_fmri_p_detrended = (pd.DataFrame(fc.partial_partial_corr(data_fmri_detrended,alpha)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.partial_partial_corr(data_fmri_detrended,alpha)[1], 
                                          index=labels, columns=labels))
max_partial_partial_fmri_detrended = np.max(fc.partial_partial_corr(data_fmri_detrended,alpha)[0][np.triu_indices(m,1)])

In [ ]:
# Partial correlation:
alpha = 1.0
partial_fmri_detrended, partial_fmri_p_detrended = (pd.DataFrame(fc.partial_partial_corr(data_fmri_detrended,alpha)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.partial_partial_corr(data_fmri_detrended,alpha)[1], 
                                          index=labels, columns=labels))
max_partial_fmri_detrended = np.max(fc.partial_partial_corr(data_fmri_detrended,alpha)[0][np.triu_indices(m,1)])

In [ ]:
# Mutual Information:
# take number of bins as square root of the data length:
nbins = int(np.floor(np.sqrt(n)))
mi_fmri_detrended = np.zeros((m,m))
for i in range(m):
    mi_fmri_detrended[i, i] = 10
    for j in range(i+1, m):
        mi_fmri_detrended[i,j] = fc.calc_MI(data_fmri_detrended[:,i], data_fmri_detrended[:,j], nbins)
        mi_fmri_detrended[j,i] = mi_fmri_detrended[i,j]

max_mi_fmri_detrended = np.max(mi_fmri_detrended[np.triu_indices(m,1)])
mi_fmri_detrended = pd.DataFrame(mi_fmri_detrended, index=labels, columns=labels)

In [ ]:
# inverse of Euclidean distance:
inv_ed_fmri_detrended = np.zeros((m,m))
for i in range(m):
    inv_ed_fmri_detrended[i, i] = 1.0
    for j in range(i+1, m):
        inv_ed_fmri_detrended[i,j] = 1.0/fc.euclidean_distance(data_fmri_detrended[:,i], data_fmri_detrended[:,j])
        inv_ed_fmri_detrended[j,i] = inv_ed_fmri_detrended[i,j]
        
max_inv_ed_fmri_detrended = np.max(inv_ed_fmri_detrended[np.triu_indices(m,1)])
inv_ed_fmri_detrended = pd.DataFrame(inv_ed_fmri_detrended, index=labels, columns=labels)

In [ ]:
# inverse of Manhattan distance:
inv_md_fmri_detrended = np.zeros((m,m))
for i in range(m):
    inv_md_fmri_detrended[i, i] = 1.0
    for j in range(i+1, m):
        inv_md_fmri_detrended[i,j] = 1.0/fc.manhattan_distance(data_fmri_detrended[:,i], data_fmri_detrended[:,j])
        inv_md_fmri_detrended[j,i] = inv_md_fmri_detrended[i,j]
        
max_inv_md_fmri_detrended = np.max(inv_md_fmri_detrended[np.triu_indices(m,1)])
inv_md_fmri_detrended = pd.DataFrame(inv_md_fmri_detrended, index=labels, columns=labels)

In [ ]:
corr_fmri_detrended = plot_corr(pearson_fmri_detrended, pearson_fmri_p_detrended, partial_partial_fmri_detrended, partial_partial_fmri_p_detrended, partial_fmri_detrended, partial_fmri_p_detrended)

In [ ]:
all_fmri_detrended = plot_all(pearson_fmri_detrended, partial_partial_fmri_detrended, partial_fmri_detrended, mi_fmri_detrended, inv_ed_fmri_detrended, inv_md_fmri_detrended, max_pearson_fmri_detrended, max_partial_partial_fmri_detrended, max_partial_fmri_detrended, max_mi_fmri_detrended, max_inv_ed_fmri_detrended, max_inv_md_fmri_detrended)

In [ ]:
folder_output = output_fMRI
fname = folder_output + 'all_methods_fmri_detrended_visual_system_subject' + subject_label

dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

hp.savef(all_fmri_detrended, fname, dpis, formats)

In [ ]:
# plot and save all the clustermaps:
folder_output = output_fMRI
dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

# Pearson correlation:
fig1 = plt.figure(figsize=(12,11))
s1 = sns.clustermap(pearson_fmri_detrended, vmin=-abs(max_pearson_fmri_detrended), vmax=abs(max_pearson_fmri_detrended), cmap='coolwarm');
plt.setp(s1.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s1.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Pearson correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_pearson_fmri_detrended_visual_system_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Partial partial correlation:
fig2 = plt.figure(figsize=(12,11))
s2 = sns.clustermap(partial_partial_fmri_detrended, vmin=-abs(max_partial_partial_fmri_detrended), vmax=abs(max_partial_partial_fmri_detrended), cmap='coolwarm');
plt.setp(s2.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s2.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Partial partial correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_partial_partial_fmri_detrended_visual_system_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Partial correlation: 
fig3 = plt.figure(figsize=(12,11))
s3 = sns.clustermap(partial_fmri_detrended, vmin=-abs(max_partial_fmri_detrended), vmax=abs(max_partial_fmri_detrended), cmap='coolwarm');
plt.setp(s3.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s3.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Partial correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_partial_fmri_detrended_visual_system_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Mutual information: 
fig4 = plt.figure(figsize=(12,11))
s4 = sns.clustermap(mi_fmri_detrended, vmin=-abs(max_mi_fmri_detrended), vmax=abs(max_mi_fmri_detrended), cmap='coolwarm');
plt.setp(s4.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s4.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Mutual information', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_mi_fmri_detrended_visual_system_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

# Euclidean distance: 
fig5 = plt.figure(figsize=(12,11))
s5 = sns.clustermap(inv_ed_fmri_detrended, vmin=-abs(max_inv_ed_fmri_detrended), vmax=abs(max_inv_ed_fmri_detrended), cmap='coolwarm');
plt.setp(s5.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s5.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Inverse Euclidean distance', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_inv_ed_fmri_detrended_visual_system_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

# Manhattan distance: 
fig6 = plt.figure(figsize=(12,11))
s6 = sns.clustermap(inv_md_fmri_detrended, vmin=-abs(max_inv_md_fmri_detrended), vmax=abs(max_inv_md_fmri_detrended), cmap='coolwarm');
plt.setp(s6.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s6.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Inverse Manhattan distance', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_inv_md_fmri_detrended_visual_system_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

In [ ]:
# plot correlation between the 12 methods:
nl = len(np.triu_indices(m,1)[0])

pearson_fmri_vectorized = np.ndarray.reshape(np.asarray(pearson_fmri)[np.triu_indices(m,1)], nl, 1)
partial_partial_fmri_vectorized = np.ndarray.reshape(np.asarray(partial_partial_fmri)[np.triu_indices(m,1)], nl, 1)
partial_fmri_vectorized = np.ndarray.reshape(np.asarray(partial_fmri)[np.triu_indices(m,1)], nl, 1)
mi_fmri_vectorized = np.ndarray.reshape(np.asarray(mi_fmri)[np.triu_indices(m,1)], nl, 1)
inv_ed_fmri_vectorized = np.ndarray.reshape(np.asarray(inv_ed_fmri)[np.triu_indices(m,1)], nl, 1)
inv_md_fmri_vectorized = np.ndarray.reshape(np.asarray(inv_md_fmri)[np.triu_indices(m,1)], nl, 1)

pearson_fmri_detrended_vectorized = np.ndarray.reshape(np.asarray(pearson_fmri_detrended)[np.triu_indices(m,1)], nl, 1)
partial_partial_fmri_detrended_vectorized = np.ndarray.reshape(np.asarray(partial_partial_fmri_detrended)[np.triu_indices(m,1)], nl, 1)
partial_fmri_detrended_vectorized = np.ndarray.reshape(np.asarray(partial_fmri_detrended)[np.triu_indices(m,1)], nl, 1)
mi_fmri_detrended_vectorized = np.ndarray.reshape(np.asarray(mi_fmri_detrended)[np.triu_indices(m,1)], nl, 1)
inv_ed_fmri_detrended_vectorized = np.ndarray.reshape(np.asarray(inv_ed_fmri_detrended)[np.triu_indices(m,1)], nl, 1)
inv_md_fmri_detrended_vectorized = np.ndarray.reshape(np.asarray(inv_md_fmri_detrended)[np.triu_indices(m,1)], nl, 1)

In [ ]:
results_allmethods = np.concatenate((pearson_fmri_vectorized, partial_partial_fmri_vectorized, partial_fmri_vectorized, \
                                     mi_fmri_vectorized, inv_ed_fmri_vectorized, inv_md_fmri_vectorized, \
                                     pearson_fmri_detrended_vectorized, partial_partial_fmri_detrended_vectorized, partial_fmri_detrended_vectorized, \
                                     mi_fmri_detrended_vectorized, inv_ed_fmri_detrended_vectorized, inv_md_fmri_detrended_vectorized),axis=1)

corr_results_allmethods, corr_results_allmethods_p = sci.stats.spearmanr(results_allmethods)
labels_allmethods = ['Pearson correlation','Partial partial correlation', 'Partial partial correlation', \
          'Mutual information', 'Inverse Euclidean distance', 'Inverse Manhattan distance', \
          'Pearson correlation, detrended data','Partial partial correlation, detrended data', 'Partial partial correlation, detrended data', \
          'Mutual information, detrended data', 'Inverse Euclidean distance, detrended data', 'Inverse Manhattan distance, detrended data']


corr_results_allmethods = pd.DataFrame(corr_results_allmethods, index=labels_allmethods, columns=labels_allmethods)
corr_results_allmethods_p = pd.DataFrame(corr_results_allmethods_p, index=labels_allmethods, columns=labels_allmethods)

fig = plt.figure(figsize=(13,10))
ax0 = plt.subplot2grid((1,2),(0,0), colspan=1, rowspan=1)
ax1 = plt.subplot2grid((1,2),(0,1), colspan=1, rowspan=1)
sns.heatmap(corr_results_allmethods, vmin=-1.0, vmax=1.0, cmap='coolwarm', ax=ax0);
ax0.set_title('Spearman r between the methods', fontsize=16,y=1.70);
sns.heatmap(corr_results_allmethods_p, vmin=0, vmax=0.01, cmap='coolwarm', ax=ax1);
ax1.set_title('Associated p values', fontsize=16,  y=1.70);
for ax in fig.axes:
    plt.sca(ax)
    ax.axvline(x=6, color='k', linestyle='-')
    ax.axhline(y=6, color='k', linestyle='-')
    ax.xaxis.tick_top()
    plt.xticks(rotation=90)
for label in ax1.get_yticklabels():
    label.set_visible(False)
plt.tight_layout()

In [ ]:
folder_output = output_fMRI
fname = folder_output + 'correlations_all_methods_fmri_visual_system_subject' + subject_label

dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

hp.savef(fig, fname, dpis, formats)

In [ ]:
# [b] choose somatosensory system:
data_fmri = data_fmri0[:,indices_somatosensory]
m = data_fmri.shape[1]
labels = labels_all[indices_somatosensory]

In [ ]:
# Pearson correlation:
pearson_fmri, pearson_fmri_p = (pd.DataFrame(fc.pearson_corr(data_fmri)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.pearson_corr(data_fmri)[1], 
                                          index=labels, columns=labels))
max_pearson_fmri = np.max(fc.pearson_corr(data_fmri)[0][np.triu_indices(m,1)])

In [ ]:
# Partial partial correlation:
alpha = 0.5
partial_partial_fmri, partial_partial_fmri_p = (pd.DataFrame(fc.partial_partial_corr(data_fmri,alpha)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.partial_partial_corr(data_fmri,alpha)[1], 
                                          index=labels, columns=labels))
max_partial_partial_fmri = np.max(fc.partial_partial_corr(data_fmri,alpha)[0][np.triu_indices(m,1)])

In [ ]:
# Partial correlation:
alpha = 1.0
partial_fmri, partial_fmri_p = (pd.DataFrame(fc.partial_partial_corr(data_fmri,alpha)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.partial_partial_corr(data_fmri,alpha)[1], 
                                          index=labels, columns=labels))
max_partial_fmri = np.max(fc.partial_partial_corr(data_fmri,alpha)[0][np.triu_indices(m,1)])

In [ ]:
# Mutual Information:
# take number of bins as square root of the data length:
nbins = int(np.floor(np.sqrt(n)))
mi_fmri = np.zeros((m,m))
for i in range(m):
    mi_fmri[i, i] = 10
    for j in range(i+1, m):
        mi_fmri[i,j] = fc.calc_MI(data_fmri[:,i], data_fmri[:,j], nbins)
        mi_fmri[j,i] = mi_fmri[i,j]

max_mi_fmri = np.max(mi_fmri[np.triu_indices(m,1)])
mi_fmri = pd.DataFrame(mi_fmri, index=labels, columns=labels)

In [ ]:
# inverse of Euclidean distance:
inv_ed_fmri = np.zeros((m,m))
for i in range(m):
    inv_ed_fmri[i, i] = 1.0
    for j in range(i+1, m):
        inv_ed_fmri[i,j] = 1.0/fc.euclidean_distance(data_fmri[:,i], data_fmri[:,j])
        inv_ed_fmri[j,i] = inv_ed_fmri[i,j]
        
max_inv_ed_fmri = np.max(inv_ed_fmri[np.triu_indices(m,1)])
inv_ed_fmri = pd.DataFrame(inv_ed_fmri, index=labels, columns=labels)

In [ ]:
# inverse of Manhattan distance:
inv_md_fmri = np.zeros((m,m))
for i in range(m):
    inv_md_fmri[i, i] = 1.0
    for j in range(i+1, m):
        inv_md_fmri[i,j] = 1.0/fc.manhattan_distance(data_fmri[:,i], data_fmri[:,j])
        inv_md_fmri[j,i] = inv_md_fmri[i,j]
        
max_inv_md_fmri = np.max(inv_md_fmri[np.triu_indices(m,1)])
inv_md_fmri = pd.DataFrame(inv_md_fmri, index=labels, columns=labels)

In [ ]:
corr_fmri = plot_corr(pearson_fmri, pearson_fmri_p, partial_partial_fmri, partial_partial_fmri_p, partial_fmri, partial_fmri_p)

In [ ]:
all_fmri = plot_all(pearson_fmri, partial_partial_fmri, partial_fmri, mi_fmri, inv_ed_fmri, inv_md_fmri, max_pearson_fmri, max_partial_partial_fmri, max_partial_fmri, max_mi_fmri, max_inv_ed_fmri, max_inv_md_fmri)

In [ ]:
folder_output = output_fMRI
fname = folder_output + 'all_methods_fmri_somatosensory_system_subject' + subject_label

dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

hp.savef(all_fmri, fname, dpis, formats)

In [ ]:
# plot and save all the clustermaps:
folder_output = output_fMRI
dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

# Pearson correlation:
fig1 = plt.figure(figsize=(12,11))
s1 = sns.clustermap(pearson_fmri, vmin=-abs(max_pearson_fmri), vmax=abs(max_pearson_fmri), cmap='coolwarm');
plt.setp(s1.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s1.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Pearson correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_pearson_fmri_somatosensory_system_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Partial partial correlation:
fig2 = plt.figure(figsize=(12,11))
s2 = sns.clustermap(partial_partial_fmri, vmin=-abs(max_partial_partial_fmri), vmax=abs(max_partial_partial_fmri), cmap='coolwarm');
plt.setp(s2.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s2.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Partial partial correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_partial_partial_fmri_somatosensory_system_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Partial correlation: 
fig3 = plt.figure(figsize=(12,11))
s3 = sns.clustermap(partial_fmri, vmin=-abs(max_partial_fmri), vmax=abs(max_partial_fmri), cmap='coolwarm');
plt.setp(s3.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s3.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Partial correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_partial_fmri_somatosensory_system_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Mutual information: 
fig4 = plt.figure(figsize=(12,11))
s4 = sns.clustermap(mi_fmri, vmin=-abs(max_mi_fmri), vmax=abs(max_mi_fmri), cmap='coolwarm');
plt.setp(s4.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s4.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Mutual information', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_mi_fmri_somatosensory_system_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

# Euclidean distance: 
fig5 = plt.figure(figsize=(12,11))
s5 = sns.clustermap(inv_ed_fmri, vmin=-abs(max_inv_ed_fmri), vmax=abs(max_inv_ed_fmri), cmap='coolwarm');
plt.setp(s5.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s5.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Inverse Euclidean distance', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_inv_ed_fmri_somatosensory_system_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

# Manhattan distance: 
fig6 = plt.figure(figsize=(12,11))
s6 = sns.clustermap(inv_md_fmri, vmin=-abs(max_inv_md_fmri), vmax=abs(max_inv_md_fmri), cmap='coolwarm');
plt.setp(s6.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s6.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Inverse Manhattan distance', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_inv_md_fmri_somatosensory_system_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

In [ ]:
# add detrending:
data_fmri_detrended = fc.detrend(data_fmri)

In [ ]:
# Pearson correlation:
pearson_fmri_detrended, pearson_fmri_p_detrended = (pd.DataFrame(fc.pearson_corr(data_fmri_detrended)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.pearson_corr(data_fmri_detrended)[1], 
                                          index=labels, columns=labels))
max_pearson_fmri_detrended = np.max(fc.pearson_corr(data_fmri_detrended)[0][np.triu_indices(m,1)])

In [ ]:
# Partial partial correlation:
alpha = 0.5
partial_partial_fmri_detrended, partial_partial_fmri_p_detrended = (pd.DataFrame(fc.partial_partial_corr(data_fmri_detrended,alpha)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.partial_partial_corr(data_fmri_detrended,alpha)[1], 
                                          index=labels, columns=labels))
max_partial_partial_fmri_detrended = np.max(fc.partial_partial_corr(data_fmri_detrended,alpha)[0][np.triu_indices(m,1)])

In [ ]:
# Partial correlation:
alpha = 1.0
partial_fmri_detrended, partial_fmri_p_detrended = (pd.DataFrame(fc.partial_partial_corr(data_fmri_detrended,alpha)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.partial_partial_corr(data_fmri_detrended,alpha)[1], 
                                          index=labels, columns=labels))
max_partial_fmri_detrended = np.max(fc.partial_partial_corr(data_fmri_detrended,alpha)[0][np.triu_indices(m,1)])

In [ ]:
# Mutual Information:
# take number of bins as square root of the data length:
nbins = int(np.floor(np.sqrt(n)))
mi_fmri_detrended = np.zeros((m,m))
for i in range(m):
    mi_fmri_detrended[i, i] = 10
    for j in range(i+1, m):
        mi_fmri_detrended[i,j] = fc.calc_MI(data_fmri_detrended[:,i], data_fmri_detrended[:,j], nbins)
        mi_fmri_detrended[j,i] = mi_fmri_detrended[i,j]

max_mi_fmri_detrended = np.max(mi_fmri_detrended[np.triu_indices(m,1)])
mi_fmri_detrended = pd.DataFrame(mi_fmri_detrended, index=labels, columns=labels)

In [ ]:
# inverse of Euclidean distance:
inv_ed_fmri_detrended = np.zeros((m,m))
for i in range(m):
    inv_ed_fmri_detrended[i, i] = 1.0
    for j in range(i+1, m):
        inv_ed_fmri_detrended[i,j] = 1.0/fc.euclidean_distance(data_fmri_detrended[:,i], data_fmri_detrended[:,j])
        inv_ed_fmri_detrended[j,i] = inv_ed_fmri_detrended[i,j]
        
max_inv_ed_fmri_detrended = np.max(inv_ed_fmri_detrended[np.triu_indices(m,1)])
inv_ed_fmri_detrended = pd.DataFrame(inv_ed_fmri_detrended, index=labels, columns=labels)

In [ ]:
# inverse of Manhattan distance:
inv_md_fmri_detrended = np.zeros((m,m))
for i in range(m):
    inv_md_fmri_detrended[i, i] = 1.0
    for j in range(i+1, m):
        inv_md_fmri_detrended[i,j] = 1.0/fc.manhattan_distance(data_fmri_detrended[:,i], data_fmri_detrended[:,j])
        inv_md_fmri_detrended[j,i] = inv_md_fmri_detrended[i,j]
        
max_inv_md_fmri_detrended = np.max(inv_md_fmri_detrended[np.triu_indices(m,1)])
inv_md_fmri_detrended = pd.DataFrame(inv_md_fmri_detrended, index=labels, columns=labels)

In [ ]:
corr_fmri_detrended = plot_corr(pearson_fmri_detrended, pearson_fmri_p_detrended, partial_partial_fmri_detrended, partial_partial_fmri_p_detrended, partial_fmri_detrended, partial_fmri_p_detrended)

In [ ]:
all_fmri_detrended = plot_all(pearson_fmri_detrended, partial_partial_fmri_detrended, partial_fmri_detrended, mi_fmri_detrended, inv_ed_fmri_detrended, inv_md_fmri_detrended, max_pearson_fmri_detrended, max_partial_partial_fmri_detrended, max_partial_fmri_detrended, max_mi_fmri_detrended, max_inv_ed_fmri_detrended, max_inv_md_fmri_detrended)

In [ ]:
folder_output = output_fMRI
fname = folder_output + 'all_methods_fmri_detrended_somatosensory_system_subject' + subject_label

dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

hp.savef(all_fmri_detrended, fname, dpis, formats)

In [ ]:
# plot and save all the clustermaps:
folder_output = output_fMRI
dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

# Pearson correlation:
fig1 = plt.figure(figsize=(12,11))
s1 = sns.clustermap(pearson_fmri_detrended, vmin=-abs(max_pearson_fmri_detrended), vmax=abs(max_pearson_fmri_detrended), cmap='coolwarm');
plt.setp(s1.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s1.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Pearson correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_pearson_fmri_detrended_somatosensory_system_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Partial partial correlation:
fig2 = plt.figure(figsize=(12,11))
s2 = sns.clustermap(partial_partial_fmri_detrended, vmin=-abs(max_partial_partial_fmri_detrended), vmax=abs(max_partial_partial_fmri_detrended), cmap='coolwarm');
plt.setp(s2.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s2.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Partial partial correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_partial_partial_fmri_detrended_somatosensory_system_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Partial correlation: 
fig3 = plt.figure(figsize=(12,11))
s3 = sns.clustermap(partial_fmri_detrended, vmin=-abs(max_partial_fmri_detrended), vmax=abs(max_partial_fmri_detrended), cmap='coolwarm');
plt.setp(s3.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s3.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Partial correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_partial_fmri_detrended_somatosensory_system_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Mutual information: 
fig4 = plt.figure(figsize=(12,11))
s4 = sns.clustermap(mi_fmri_detrended, vmin=-abs(max_mi_fmri_detrended), vmax=abs(max_mi_fmri_detrended), cmap='coolwarm');
plt.setp(s4.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s4.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Mutual information', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_mi_fmri_detrended_somatosensory_system_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

# Euclidean distance: 
fig5 = plt.figure(figsize=(12,11))
s5 = sns.clustermap(inv_ed_fmri_detrended, vmin=-abs(max_inv_ed_fmri_detrended), vmax=abs(max_inv_ed_fmri_detrended), cmap='coolwarm');
plt.setp(s5.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s5.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Inverse Euclidean distance', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_inv_ed_fmri_detrended_somatosensory_system_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

# Manhattan distance: 
fig6 = plt.figure(figsize=(12,11))
s6 = sns.clustermap(inv_md_fmri_detrended, vmin=-abs(max_inv_md_fmri_detrended), vmax=abs(max_inv_md_fmri_detrended), cmap='coolwarm');
plt.setp(s6.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s6.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Inverse Manhattan distance', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_inv_md_fmri_detrended_somatosensory_system_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

In [ ]:
# plot correlation between the 12 methods:
nl = len(np.triu_indices(m,1)[0])

pearson_fmri_vectorized = np.ndarray.reshape(np.asarray(pearson_fmri)[np.triu_indices(m,1)], nl, 1)
partial_partial_fmri_vectorized = np.ndarray.reshape(np.asarray(partial_partial_fmri)[np.triu_indices(m,1)], nl, 1)
partial_fmri_vectorized = np.ndarray.reshape(np.asarray(partial_fmri)[np.triu_indices(m,1)], nl, 1)
mi_fmri_vectorized = np.ndarray.reshape(np.asarray(mi_fmri)[np.triu_indices(m,1)], nl, 1)
inv_ed_fmri_vectorized = np.ndarray.reshape(np.asarray(inv_ed_fmri)[np.triu_indices(m,1)], nl, 1)
inv_md_fmri_vectorized = np.ndarray.reshape(np.asarray(inv_md_fmri)[np.triu_indices(m,1)], nl, 1)

pearson_fmri_detrended_vectorized = np.ndarray.reshape(np.asarray(pearson_fmri_detrended)[np.triu_indices(m,1)], nl, 1)
partial_partial_fmri_detrended_vectorized = np.ndarray.reshape(np.asarray(partial_partial_fmri_detrended)[np.triu_indices(m,1)], nl, 1)
partial_fmri_detrended_vectorized = np.ndarray.reshape(np.asarray(partial_fmri_detrended)[np.triu_indices(m,1)], nl, 1)
mi_fmri_detrended_vectorized = np.ndarray.reshape(np.asarray(mi_fmri_detrended)[np.triu_indices(m,1)], nl, 1)
inv_ed_fmri_detrended_vectorized = np.ndarray.reshape(np.asarray(inv_ed_fmri_detrended)[np.triu_indices(m,1)], nl, 1)
inv_md_fmri_detrended_vectorized = np.ndarray.reshape(np.asarray(inv_md_fmri_detrended)[np.triu_indices(m,1)], nl, 1)

In [ ]:
results_allmethods = np.concatenate((pearson_fmri_vectorized, partial_partial_fmri_vectorized, partial_fmri_vectorized, \
                                     mi_fmri_vectorized, inv_ed_fmri_vectorized, inv_md_fmri_vectorized, \
                                     pearson_fmri_detrended_vectorized, partial_partial_fmri_detrended_vectorized, partial_fmri_detrended_vectorized, \
                                     mi_fmri_detrended_vectorized, inv_ed_fmri_detrended_vectorized, inv_md_fmri_detrended_vectorized),axis=1)

corr_results_allmethods, corr_results_allmethods_p = sci.stats.spearmanr(results_allmethods)
labels_allmethods = ['Pearson correlation','Partial partial correlation', 'Partial partial correlation', \
          'Mutual information', 'Inverse Euclidean distance', 'Inverse Manhattan distance', \
          'Pearson correlation, detrended data','Partial partial correlation, detrended data', 'Partial partial correlation, detrended data', \
          'Mutual information, detrended data', 'Inverse Euclidean distance, detrended data', 'Inverse Manhattan distance, detrended data']


corr_results_allmethods = pd.DataFrame(corr_results_allmethods, index=labels_allmethods, columns=labels_allmethods)
corr_results_allmethods_p = pd.DataFrame(corr_results_allmethods_p, index=labels_allmethods, columns=labels_allmethods)

fig = plt.figure(figsize=(13,10))
ax0 = plt.subplot2grid((1,2),(0,0), colspan=1, rowspan=1)
ax1 = plt.subplot2grid((1,2),(0,1), colspan=1, rowspan=1)
sns.heatmap(corr_results_allmethods, vmin=-1.0, vmax=1.0, cmap='coolwarm', ax=ax0);
ax0.set_title('Spearman r between the methods', fontsize=16,y=1.70);
sns.heatmap(corr_results_allmethods_p, vmin=0, vmax=0.01, cmap='coolwarm', ax=ax1);
ax1.set_title('Associated p values', fontsize=16,  y=1.70);
for ax in fig.axes:
    plt.sca(ax)
    ax.axvline(x=6, color='k', linestyle='-')
    ax.axhline(y=6, color='k', linestyle='-')
    ax.xaxis.tick_top()
    plt.xticks(rotation=90)
for label in ax1.get_yticklabels():
    label.set_visible(False)
plt.tight_layout()

In [ ]:
folder_output = output_fMRI
fname = folder_output + 'correlations_all_methods_fmri_somatosensory_system_subject' + subject_label

dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

hp.savef(fig, fname, dpis, formats)

## Example on MEG data

In [ ]:
subject_label = '100307'
path = foldername_data_MEG + subject_label + '.csv'

In [ ]:
data_meg = pd.read_csv(path, header=0)
labels = data_meg.columns
data_meg = np.asarray(data_meg)

# Choose a subset of regions to limit the network size (provisoric version):
Nreg = 20
labels = labels[:Nreg]
data_meg = data_meg[:,:Nreg]

n = data_meg.shape[0]
m = data_meg.shape[1]

In [ ]:
# Pearson correlation:
pearson_meg, pearson_meg_p = (pd.DataFrame(fc.pearson_corr(data_meg)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.pearson_corr(data_meg)[1], 
                                          index=labels, columns=labels))
max_pearson_meg = np.max(fc.pearson_corr(data_meg)[0][np.triu_indices(m,1)])

In [ ]:
# Partial partial correlation:
alpha = 0.5
partial_partial_meg, partial_partial_meg_p = (pd.DataFrame(fc.partial_partial_corr(data_meg,alpha)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.partial_partial_corr(data_meg,alpha)[1], 
                                          index=labels, columns=labels))
max_partial_partial_meg = np.max(fc.partial_partial_corr(data_meg,alpha)[0][np.triu_indices(m,1)])

In [ ]:
# Partial correlation:
alpha = 1.0
partial_meg, partial_meg_p = (pd.DataFrame(fc.partial_partial_corr(data_meg,alpha)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.partial_partial_corr(data_meg,alpha)[1], 
                                          index=labels, columns=labels))
max_partial_meg = np.max(fc.partial_partial_corr(data_meg,alpha)[0][np.triu_indices(m,1)])

In [ ]:
# Mutual Information:
# take number of bins as square root of the data length:
nbins = int(np.floor(np.sqrt(n)))
mi_meg = np.zeros((m,m))
for i in range(m):
    mi_meg[i, i] = 10
    for j in range(i+1, m):
        mi_meg[i,j] = fc.calc_MI(data_meg[:,i], data_meg[:,j], nbins)
        mi_meg[j,i] = mi_meg[i,j]

max_mi_meg = np.max(mi_meg[np.triu_indices(m,1)])
mi_meg = pd.DataFrame(mi_meg, index=labels, columns=labels)

In [ ]:
# inverse of Euclidean distance:
inv_ed_meg = np.zeros((m,m))
for i in range(m):
    inv_ed_meg[i, i] = 1.0
    for j in range(i+1, m):
        inv_ed_meg[i,j] = 1.0/fc.euclidean_distance(data_meg[:,i], data_meg[:,j])
        inv_ed_meg[j,i] = inv_ed_meg[i,j]
        
max_inv_ed_meg = np.max(inv_ed_meg[np.triu_indices(m,1)])
inv_ed_meg = pd.DataFrame(inv_ed_meg, index=labels, columns=labels)

In [ ]:
# inverse of Manhattan distance:
inv_md_meg = np.zeros((m,m))
for i in range(m):
    inv_md_meg[i, i] = 1.0
    for j in range(i+1, m):
        inv_md_meg[i,j] = 1.0/fc.manhattan_distance(data_meg[:,i], data_meg[:,j])
        inv_md_meg[j,i] = inv_md_meg[i,j]
        
max_inv_md_meg = np.max(inv_md_meg[np.triu_indices(m,1)])
inv_md_meg = pd.DataFrame(inv_md_meg, index=labels, columns=labels)

In [ ]:
corr_meg = plot_corr(pearson_meg, pearson_meg_p, partial_partial_meg, partial_partial_meg_p, partial_meg, partial_meg_p)

In [ ]:
all_meg = plot_all(pearson_meg, partial_partial_meg, partial_meg, mi_meg, inv_ed_meg, inv_md_meg, max_pearson_meg, max_partial_partial_meg, max_partial_meg, max_mi_meg, max_inv_ed_meg, max_inv_md_meg)

In [ ]:
folder_output = output_MEG
fname = folder_output + 'all_methods_meg_subject' + subject_label

dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

hp.savef(all_meg, fname, dpis, formats)

In [ ]:
# plot and save all the clustermaps:
folder_output = output_MEG
dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

# Pearson correlation:
fig1 = plt.figure(figsize=(12,11))
s1 = sns.clustermap(pearson_meg, vmin=-abs(max_pearson_meg), vmax=abs(max_pearson_meg), cmap='coolwarm');
plt.setp(s1.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s1.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Pearson correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_pearson_meg_system_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Partial partial correlation:
fig2 = plt.figure(figsize=(12,11))
s2 = sns.clustermap(partial_partial_meg, vmin=-abs(max_partial_partial_meg), vmax=abs(max_partial_partial_meg), cmap='coolwarm');
plt.setp(s2.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s2.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Partial partial correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_partial_partial_meg_system_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Partial correlation: 
fig3 = plt.figure(figsize=(12,11))
s3 = sns.clustermap(partial_meg, vmin=-abs(max_partial_meg), vmax=abs(max_partial_meg), cmap='coolwarm');
plt.setp(s3.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s3.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Partial correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_partial_meg_system_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Mutual information: 
fig4 = plt.figure(figsize=(12,11))
s4 = sns.clustermap(mi_meg, vmin=-abs(max_mi_meg), vmax=abs(max_mi_meg), cmap='coolwarm');
plt.setp(s4.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s4.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Mutual information', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_mi_meg_system_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

# Euclidean distance: 
fig5 = plt.figure(figsize=(12,11))
s5 = sns.clustermap(inv_ed_meg, vmin=-abs(max_inv_ed_meg), vmax=abs(max_inv_ed_meg), cmap='coolwarm');
plt.setp(s5.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s5.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Inverse Euclidean distance', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_inv_ed_meg_system_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

# Manhattan distance: 
fig6 = plt.figure(figsize=(12,11))
s6 = sns.clustermap(inv_md_meg, vmin=-abs(max_inv_md_meg), vmax=abs(max_inv_md_meg), cmap='coolwarm');
plt.setp(s6.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s6.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Inverse Manhattan distance', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_inv_md_meg_system_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

In [ ]:
# add detrending:
data_meg_detrended = fc.detrend(data_meg)

In [ ]:
# Pearson correlation:
pearson_meg_detrended, pearson_meg_p_detrended = (pd.DataFrame(fc.pearson_corr(data_meg_detrended)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.pearson_corr(data_meg_detrended)[1], 
                                          index=labels, columns=labels))
max_pearson_meg_detrended = np.max(fc.pearson_corr(data_meg_detrended)[0][np.triu_indices(m,1)])

In [ ]:
# Partial partial correlation:
alpha = 0.5
partial_partial_meg_detrended, partial_partial_meg_p_detrended = (pd.DataFrame(fc.partial_partial_corr(data_meg_detrended,alpha)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.partial_partial_corr(data_meg_detrended,alpha)[1], 
                                          index=labels, columns=labels))
max_partial_partial_meg_detrended = np.max(fc.partial_partial_corr(data_meg_detrended,alpha)[0][np.triu_indices(m,1)])

In [ ]:
# Partial correlation:
alpha = 1.0
partial_meg_detrended, partial_meg_p_detrended = (pd.DataFrame(fc.partial_partial_corr(data_meg_detrended,alpha)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.partial_partial_corr(data_meg_detrended,alpha)[1], 
                                          index=labels, columns=labels))
max_partial_meg_detrended = np.max(fc.partial_partial_corr(data_meg_detrended,alpha)[0][np.triu_indices(m,1)])

In [ ]:
# Mutual Information:
# take number of bins as square root of the data length:
nbins = int(np.floor(np.sqrt(n)))
mi_meg_detrended = np.zeros((m,m))
for i in range(m):
    mi_meg_detrended[i, i] = 10
    for j in range(i+1, m):
        mi_meg_detrended[i,j] = fc.calc_MI(data_meg_detrended[:,i], data_meg_detrended[:,j], nbins)
        mi_meg_detrended[j,i] = mi_meg_detrended[i,j]

max_mi_meg_detrended = np.max(mi_meg_detrended[np.triu_indices(m,1)])
mi_meg_detrended = pd.DataFrame(mi_meg_detrended, index=labels, columns=labels)

In [ ]:
# inverse of Euclidean distance:
inv_ed_meg_detrended = np.zeros((m,m))
for i in range(m):
    inv_ed_meg_detrended[i, i] = 1.0
    for j in range(i+1, m):
        inv_ed_meg_detrended[i,j] = 1.0/fc.euclidean_distance(data_meg_detrended[:,i], data_meg_detrended[:,j])
        inv_ed_meg_detrended[j,i] = inv_ed_meg_detrended[i,j]
        
max_inv_ed_meg_detrended = np.max(inv_ed_meg_detrended[np.triu_indices(m,1)])
inv_ed_meg_detrended = pd.DataFrame(inv_ed_meg_detrended, index=labels, columns=labels)

In [ ]:
# inverse of Manhattan distance:
inv_md_meg_detrended = np.zeros((m,m))
for i in range(m):
    inv_md_meg_detrended[i, i] = 1.0
    for j in range(i+1, m):
        inv_md_meg_detrended[i,j] = 1.0/fc.manhattan_distance(data_meg_detrended[:,i], data_meg_detrended[:,j])
        inv_md_meg_detrended[j,i] = inv_md_meg_detrended[i,j]
        
max_inv_md_meg_detrended = np.max(inv_md_meg_detrended[np.triu_indices(m,1)])
inv_md_meg_detrended = pd.DataFrame(inv_md_meg_detrended, index=labels, columns=labels)

In [ ]:
corr_meg_detrended = plot_corr(pearson_meg_detrended, pearson_meg_p_detrended, partial_partial_meg_detrended, partial_partial_meg_p_detrended, partial_meg_detrended, partial_meg_p_detrended)

In [ ]:
all_meg_detrended = plot_all(pearson_meg_detrended, partial_partial_meg_detrended, partial_meg_detrended, mi_meg_detrended, inv_ed_meg_detrended, inv_md_meg_detrended, max_pearson_meg_detrended, max_partial_partial_meg_detrended, max_partial_meg_detrended, max_mi_meg_detrended, max_inv_ed_meg_detrended, max_inv_md_meg_detrended)

In [ ]:
folder_output = output_MEG
fname = folder_output + 'all_methods_meg_detrended_subject' + subject_label

dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

hp.savef(all_meg_detrended, fname, dpis, formats)

In [ ]:
# plot and save all the clustermaps:
folder_output = output_MEG
dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

# Pearson correlation:
fig1 = plt.figure(figsize=(12,11))
s1 = sns.clustermap(pearson_meg_detrended, vmin=-abs(max_pearson_meg_detrended), vmax=abs(max_pearson_meg_detrended), cmap='coolwarm');
plt.setp(s1.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s1.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Pearson correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_pearson_meg_detrended_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Partial partial correlation:
fig2 = plt.figure(figsize=(12,11))
s2 = sns.clustermap(partial_partial_meg_detrended, vmin=-abs(max_partial_partial_meg_detrended), vmax=abs(max_partial_partial_meg_detrended), cmap='coolwarm');
plt.setp(s2.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s2.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Partial partial correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_partial_partial_meg_detrended_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Partial correlation: 
fig3 = plt.figure(figsize=(12,11))
s3 = sns.clustermap(partial_meg_detrended, vmin=-abs(max_partial_meg_detrended), vmax=abs(max_partial_meg_detrended), cmap='coolwarm');
plt.setp(s3.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s3.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Partial correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_partial_meg_detrended_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Mutual information: 
fig4 = plt.figure(figsize=(12,11))
s4 = sns.clustermap(mi_meg_detrended, vmin=-abs(max_mi_meg_detrended), vmax=abs(max_mi_meg_detrended), cmap='coolwarm');
plt.setp(s4.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s4.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Mutual information', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_mi_meg_detrended_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

# Euclidean distance: 
fig5 = plt.figure(figsize=(12,11))
s5 = sns.clustermap(inv_ed_meg_detrended, vmin=-abs(max_inv_ed_meg_detrended), vmax=abs(max_inv_ed_meg_detrended), cmap='coolwarm');
plt.setp(s5.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s5.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Inverse Euclidean distance', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_inv_ed_meg_detrended_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

# Manhattan distance: 
fig6 = plt.figure(figsize=(12,11))
s6 = sns.clustermap(inv_md_meg_detrended, vmin=-abs(max_inv_md_meg_detrended), vmax=abs(max_inv_md_meg_detrended), cmap='coolwarm');
plt.setp(s6.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s6.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Inverse Manhattan distance', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_inv_md_meg_detrended_subject' + \
        subject_label + '_' + str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

In [ ]:
# plot correlation between the 12 methods:
nl = len(np.triu_indices(m,1)[0])

pearson_meg_vectorized = np.ndarray.reshape(np.asarray(pearson_meg)[np.triu_indices(m,1)], nl, 1)
partial_partial_meg_vectorized = np.ndarray.reshape(np.asarray(partial_partial_meg)[np.triu_indices(m,1)], nl, 1)
partial_meg_vectorized = np.ndarray.reshape(np.asarray(partial_meg)[np.triu_indices(m,1)], nl, 1)
mi_meg_vectorized = np.ndarray.reshape(np.asarray(mi_meg)[np.triu_indices(m,1)], nl, 1)
inv_ed_meg_vectorized = np.ndarray.reshape(np.asarray(inv_ed_meg)[np.triu_indices(m,1)], nl, 1)
inv_md_meg_vectorized = np.ndarray.reshape(np.asarray(inv_md_meg)[np.triu_indices(m,1)], nl, 1)

pearson_meg_detrended_vectorized = np.ndarray.reshape(np.asarray(pearson_meg_detrended)[np.triu_indices(m,1)], nl, 1)
partial_partial_meg_detrended_vectorized = np.ndarray.reshape(np.asarray(partial_partial_meg_detrended)[np.triu_indices(m,1)], nl, 1)
partial_meg_detrended_vectorized = np.ndarray.reshape(np.asarray(partial_meg_detrended)[np.triu_indices(m,1)], nl, 1)
mi_meg_detrended_vectorized = np.ndarray.reshape(np.asarray(mi_meg_detrended)[np.triu_indices(m,1)], nl, 1)
inv_ed_meg_detrended_vectorized = np.ndarray.reshape(np.asarray(inv_ed_meg_detrended)[np.triu_indices(m,1)], nl, 1)
inv_md_meg_detrended_vectorized = np.ndarray.reshape(np.asarray(inv_md_meg_detrended)[np.triu_indices(m,1)], nl, 1)

results_allmethods = np.concatenate((pearson_meg_vectorized, partial_partial_meg_vectorized, partial_meg_vectorized, \
                                     mi_meg_vectorized, inv_ed_meg_vectorized, inv_md_meg_vectorized, \
                                     pearson_meg_detrended_vectorized, partial_partial_meg_detrended_vectorized, partial_meg_detrended_vectorized, \
                                     mi_meg_detrended_vectorized, inv_ed_meg_detrended_vectorized, inv_md_meg_detrended_vectorized),axis=1)

corr_results_allmethods, corr_results_allmethods_p = sci.stats.spearmanr(results_allmethods)
labels_allmethods = ['Pearson correlation','Partial partial correlation', 'Partial partial correlation', \
          'Inverse Mutual information', 'Inverse Euclidean distance', 'Manhattan distance', \
          'Pearson correlation, detrended data','Partial partial correlation, detrended data', 'Partial partial correlation, detrended data', \
          'Mutual information, detrended data', 'Inverse Euclidean distance, detrended data', 'Inverse Manhattan distance, detrended data']


corr_results_allmethods = pd.DataFrame(corr_results_allmethods, index=labels_allmethods, columns=labels_allmethods)
corr_results_allmethods_p = pd.DataFrame(corr_results_allmethods_p, index=labels_allmethods, columns=labels_allmethods)

fig = plt.figure(figsize=(13,10))
ax0 = plt.subplot2grid((1,2),(0,0), colspan=1, rowspan=1)
ax1 = plt.subplot2grid((1,2),(0,1), colspan=1, rowspan=1)
sns.heatmap(corr_results_allmethods, vmin=-1.0, vmax=1.0, cmap='coolwarm', ax=ax0);
ax0.set_title('Spearman r between the methods', fontsize=16,y=1.70);
sns.heatmap(corr_results_allmethods_p, vmin=0, vmax=0.01, cmap='coolwarm', ax=ax1);
ax1.set_title('Associated p values', fontsize=16,  y=1.70);
for ax in fig.axes:
    plt.sca(ax)
    ax.axvline(x=6, color='k', linestyle='-')
    ax.axhline(y=6, color='k', linestyle='-')
    ax.xaxis.tick_top()
    plt.xticks(rotation=90)
for label in ax1.get_yticklabels():
    label.set_visible(False)
plt.tight_layout()

folder_output = output_MEG
fname = folder_output + 'correlations_all_methods_meg_subject' + subject_label

dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

hp.savef(fig, fname, dpis, formats)

## Example on Google trends data

In [ ]:
path = foldername_data_google_trends + 'googletrends_allphrases_5years.csv'
data_google_trends0 = pd.read_csv(path, header=0, sep=';', index_col='Week')
labels = pd.Index.tolist(data_google_trends0.columns)
data_google_trends0 = np.asarray(data_google_trends0)

n = data_google_trends0.shape[0]
m = data_google_trends0.shape[1]

In [ ]:
# normalize the data with boxcox:
# add little number to the data (so that all the values are positive):
data_google_trends = data_google_trends0 + 1e-6*np.ones(data_google_trends0.shape)

data = data_google_trends[:,0]

for ind in range(m):
    boxcoxed = sci.stats.boxcox(np.ndarray.reshape(data_google_trends[:,ind], n, 1))
    data_google_trends[:,ind] = boxcoxed[0].reshape((n,))

In [ ]:
# Pearson correlation:
pearson_google_trends, pearson_google_trends_p = (pd.DataFrame(fc.pearson_corr(data_google_trends)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.pearson_corr(data_google_trends)[1], 
                                          index=labels, columns=labels))
max_pearson_google_trends = np.max(fc.pearson_corr(data_google_trends)[0][np.triu_indices(m,1)])

# Partial partial correlation:
alpha = 0.5
partial_partial_google_trends, partial_partial_google_trends_p = (pd.DataFrame(fc.partial_partial_corr(data_google_trends,alpha)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.partial_partial_corr(data_google_trends,alpha)[1], 
                                          index=labels, columns=labels))
max_partial_partial_google_trends = np.max(fc.partial_partial_corr(data_google_trends,alpha)[0][np.triu_indices(m,1)])

# Partial correlation:
alpha = 1.0
partial_google_trends, partial_google_trends_p = (pd.DataFrame(fc.partial_partial_corr(data_google_trends,alpha)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.partial_partial_corr(data_google_trends,alpha)[1], 
                                          index=labels, columns=labels))
max_partial_google_trends = np.max(fc.partial_partial_corr(data_google_trends,alpha)[0][np.triu_indices(m,1)])

# Mutual Information:
# take number of bins as square root of the data length:
nbins = int(np.floor(np.sqrt(n)))
mi_google_trends = np.zeros((m,m))
for i in range(m):
    mi_google_trends[i, i] = 10
    for j in range(i+1, m):
        mi_google_trends[i,j] = fc.calc_MI(data_google_trends[:,i], data_google_trends[:,j], nbins)
        mi_google_trends[j,i] = mi_google_trends[i,j]

max_mi_google_trends = np.max(mi_google_trends[np.triu_indices(m,1)])
mi_google_trends = pd.DataFrame(mi_google_trends, index=labels, columns=labels)

# inverse of Euclidean distance:
inv_ed_google_trends = np.zeros((m,m))
for i in range(m):
    inv_ed_google_trends[i, i] = 1.0
    for j in range(i+1, m):
        inv_ed_google_trends[i,j] = 1.0/fc.euclidean_distance(data_google_trends[:,i], data_google_trends[:,j])
        inv_ed_google_trends[j,i] = inv_ed_google_trends[i,j]
        
max_inv_ed_google_trends = np.max(inv_ed_google_trends[np.triu_indices(m,1)])
inv_ed_google_trends = pd.DataFrame(inv_ed_google_trends, index=labels, columns=labels)

# inverse of Manhattan distance:
inv_md_google_trends = np.zeros((m,m))
for i in range(m):
    inv_md_google_trends[i, i] = 1.0
    for j in range(i+1, m):
        inv_md_google_trends[i,j] = 1.0/fc.manhattan_distance(data_google_trends[:,i], data_google_trends[:,j])
        inv_md_google_trends[j,i] = inv_md_google_trends[i,j]
        
max_inv_md_google_trends = np.max(inv_md_google_trends[np.triu_indices(m,1)])
inv_md_google_trends = pd.DataFrame(inv_md_google_trends, index=labels, columns=labels)

corr_google_trends = plot_corr(pearson_google_trends, pearson_google_trends_p, partial_partial_google_trends, partial_partial_google_trends_p, partial_google_trends, partial_google_trends_p)

all_google_trends = plot_all(pearson_google_trends, partial_partial_google_trends, partial_google_trends, mi_google_trends, inv_ed_google_trends, inv_md_google_trends, max_pearson_google_trends, max_partial_partial_google_trends, max_partial_google_trends, max_mi_google_trends, max_inv_ed_google_trends, max_inv_md_google_trends)

folder_output = output_google_trends
fname = folder_output + 'all_methods_google_trends'

dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

hp.savef(all_google_trends, fname, dpis, formats)

# plot and save all the clustermaps:
folder_output = output_google_trends
dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

# Pearson correlation:
fig1 = plt.figure(figsize=(12,11))
s1 = sns.clustermap(pearson_google_trends, vmin=-abs(max_pearson_google_trends), vmax=abs(max_pearson_google_trends), cmap='coolwarm');
plt.setp(s1.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s1.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Pearson correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_pearson_google_trends_' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Partial partial correlation:
fig2 = plt.figure(figsize=(12,11))
s2 = sns.clustermap(partial_partial_google_trends, vmin=-abs(max_partial_partial_google_trends), vmax=abs(max_partial_partial_google_trends), cmap='coolwarm');
plt.setp(s2.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s2.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Partial partial correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_partial_partial_google_trends_' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Partial correlation: 
fig3 = plt.figure(figsize=(12,11))
s3 = sns.clustermap(partial_google_trends, vmin=-abs(max_partial_google_trends), vmax=abs(max_partial_google_trends), cmap='coolwarm');
plt.setp(s3.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s3.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Partial correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_partial_google_trends_' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Mutual information: 
fig4 = plt.figure(figsize=(12,11))
s4 = sns.clustermap(mi_google_trends, vmin=-abs(max_mi_google_trends), vmax=abs(max_mi_google_trends), cmap='coolwarm');
plt.setp(s4.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s4.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Mutual information', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_mi_google_trends_' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

# Euclidean distance: 
fig5 = plt.figure(figsize=(12,11))
s5 = sns.clustermap(inv_ed_google_trends, vmin=-abs(max_inv_ed_google_trends), vmax=abs(max_inv_ed_google_trends), cmap='coolwarm');
plt.setp(s5.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s5.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Inverse Euclidean distance', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_inv_ed_google_trends_' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

# Manhattan distance: 
fig6 = plt.figure(figsize=(12,11))
s6 = sns.clustermap(inv_md_google_trends, vmin=-abs(max_inv_md_google_trends), vmax=abs(max_inv_md_google_trends), cmap='coolwarm');
plt.setp(s6.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s6.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Inverse Manhattan distance', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_inv_md_google_trends_' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

        
        
# add detrending:
data_google_trends_detrended = fc.detrend(data_google_trends)

# Pearson correlation:
pearson_google_trends_detrended, pearson_google_trends_p_detrended = (pd.DataFrame(fc.pearson_corr(data_google_trends_detrended)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.pearson_corr(data_google_trends_detrended)[1], 
                                          index=labels, columns=labels))
max_pearson_google_trends_detrended = np.max(fc.pearson_corr(data_google_trends_detrended)[0][np.triu_indices(m,1)])

# Partial partial correlation:
alpha = 0.5
partial_partial_google_trends_detrended, partial_partial_google_trends_p_detrended = (pd.DataFrame(fc.partial_partial_corr(data_google_trends_detrended,alpha)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.partial_partial_corr(data_google_trends_detrended,alpha)[1], 
                                          index=labels, columns=labels))
max_partial_partial_google_trends_detrended = np.max(fc.partial_partial_corr(data_google_trends_detrended,alpha)[0][np.triu_indices(m,1)])

# Partial correlation:
alpha = 1.0
partial_google_trends_detrended, partial_google_trends_p_detrended = (pd.DataFrame(fc.partial_partial_corr(data_google_trends_detrended,alpha)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.partial_partial_corr(data_google_trends_detrended,alpha)[1], 
                                          index=labels, columns=labels))
max_partial_google_trends_detrended = np.max(fc.partial_partial_corr(data_google_trends_detrended,alpha)[0][np.triu_indices(m,1)])

# Mutual Information:
# take number of bins as square root of the data length:
nbins = int(np.floor(np.sqrt(n)))
mi_google_trends_detrended = np.zeros((m,m))
for i in range(m):
    mi_google_trends_detrended[i, i] = 10
    for j in range(i+1, m):
        mi_google_trends_detrended[i,j] = fc.calc_MI(data_google_trends_detrended[:,i], data_google_trends_detrended[:,j], nbins)
        mi_google_trends_detrended[j,i] = mi_google_trends_detrended[i,j]

max_mi_google_trends_detrended = np.max(mi_google_trends_detrended[np.triu_indices(m,1)])
mi_google_trends_detrended = pd.DataFrame(mi_google_trends_detrended, index=labels, columns=labels)

# inverse of Euclidean distance:
inv_ed_google_trends_detrended = np.zeros((m,m))
for i in range(m):
    inv_ed_google_trends_detrended[i, i] = 1.0
    for j in range(i+1, m):
        inv_ed_google_trends_detrended[i,j] = 1.0/fc.euclidean_distance(data_google_trends_detrended[:,i], data_google_trends_detrended[:,j])
        inv_ed_google_trends_detrended[j,i] = inv_ed_google_trends_detrended[i,j]
        
max_inv_ed_google_trends_detrended = np.max(inv_ed_google_trends_detrended[np.triu_indices(m,1)])
inv_ed_google_trends_detrended = pd.DataFrame(inv_ed_google_trends_detrended, index=labels, columns=labels)

# inverse of Manhattan distance:
inv_md_google_trends_detrended = np.zeros((m,m))
for i in range(m):
    inv_md_google_trends_detrended[i, i] = 1.0
    for j in range(i+1, m):
        inv_md_google_trends_detrended[i,j] = 1.0/fc.manhattan_distance(data_google_trends_detrended[:,i], data_google_trends_detrended[:,j])
        inv_md_google_trends_detrended[j,i] = inv_md_google_trends_detrended[i,j]
        
max_inv_md_google_trends_detrended = np.max(inv_md_google_trends_detrended[np.triu_indices(m,1)])
inv_md_google_trends_detrended = pd.DataFrame(inv_md_google_trends_detrended, index=labels, columns=labels)

corr_google_trends_detrended = plot_corr(pearson_google_trends_detrended, pearson_google_trends_p_detrended, partial_partial_google_trends_detrended, partial_partial_google_trends_p_detrended, partial_google_trends_detrended, partial_google_trends_p_detrended)

all_google_trends_detrended = plot_all(pearson_google_trends_detrended, partial_partial_google_trends_detrended, partial_google_trends_detrended, mi_google_trends_detrended, inv_ed_google_trends_detrended, inv_md_google_trends_detrended, max_pearson_google_trends_detrended, max_partial_partial_google_trends_detrended, max_partial_google_trends_detrended, max_mi_google_trends_detrended, max_inv_ed_google_trends_detrended, max_inv_md_google_trends_detrended)

folder_output = output_google_trends
fname = folder_output + 'all_methods_google_trends_detrended'

dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

hp.savef(all_google_trends_detrended, fname, dpis, formats)

# plot and save all the clustermaps:
folder_output = output_google_trends
dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

# Pearson correlation:
fig1 = plt.figure(figsize=(12,11))
s1 = sns.clustermap(pearson_google_trends_detrended, vmin=-abs(max_pearson_google_trends_detrended), vmax=abs(max_pearson_google_trends_detrended), cmap='coolwarm');
plt.setp(s1.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s1.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Pearson correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_pearson_google_trends_detrended_' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Partial partial correlation:
fig2 = plt.figure(figsize=(12,11))
s2 = sns.clustermap(partial_partial_google_trends_detrended, vmin=-abs(max_partial_partial_google_trends_detrended), vmax=abs(max_partial_partial_google_trends_detrended), cmap='coolwarm');
plt.setp(s2.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s2.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Partial partial correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_partial_partial_google_trends_detrended_' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Partial correlation: 
fig3 = plt.figure(figsize=(12,11))
s3 = sns.clustermap(partial_google_trends_detrended, vmin=-abs(max_partial_google_trends_detrended), vmax=abs(max_partial_google_trends_detrended), cmap='coolwarm');
plt.setp(s3.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s3.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Partial correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_partial_google_trends_detrended_' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Mutual information: 
fig4 = plt.figure(figsize=(12,11))
s4 = sns.clustermap(mi_google_trends_detrended, vmin=-abs(max_mi_google_trends_detrended), vmax=abs(max_mi_google_trends_detrended), cmap='coolwarm');
plt.setp(s4.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s4.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Mutual information', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_mi_google_trends_detrended' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

# Euclidean distance: 
fig5 = plt.figure(figsize=(12,11))
s5 = sns.clustermap(inv_ed_google_trends_detrended, vmin=-abs(max_inv_ed_google_trends_detrended), vmax=abs(max_inv_ed_google_trends_detrended), cmap='coolwarm');
plt.setp(s5.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s5.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Inverse Euclidean distance', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_inv_ed_google_trends_detrended' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

# Manhattan distance: 
fig6 = plt.figure(figsize=(12,11))
s6 = sns.clustermap(inv_md_google_trends_detrended, vmin=-abs(max_inv_md_google_trends_detrended), vmax=abs(max_inv_md_google_trends_detrended), cmap='coolwarm');
plt.setp(s6.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s6.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Inverse Manhattan distance', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_inv_md_google_trends_detrended' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

# plot correlation between the 12 methods:
nl = len(np.triu_indices(m,1)[0])

pearson_google_trends_vectorized = np.ndarray.reshape(np.asarray(pearson_google_trends)[np.triu_indices(m,1)], nl, 1)
partial_partial_google_trends_vectorized = np.ndarray.reshape(np.asarray(partial_partial_google_trends)[np.triu_indices(m,1)], nl, 1)
partial_google_trends_vectorized = np.ndarray.reshape(np.asarray(partial_google_trends)[np.triu_indices(m,1)], nl, 1)
mi_google_trends_vectorized = np.ndarray.reshape(np.asarray(mi_google_trends)[np.triu_indices(m,1)], nl, 1)
inv_ed_google_trends_vectorized = np.ndarray.reshape(np.asarray(inv_ed_google_trends)[np.triu_indices(m,1)], nl, 1)
inv_md_google_trends_vectorized = np.ndarray.reshape(np.asarray(inv_md_google_trends)[np.triu_indices(m,1)], nl, 1)

pearson_google_trends_detrended_vectorized = np.ndarray.reshape(np.asarray(pearson_google_trends_detrended)[np.triu_indices(m,1)], nl, 1)
partial_partial_google_trends_detrended_vectorized = np.ndarray.reshape(np.asarray(partial_partial_google_trends_detrended)[np.triu_indices(m,1)], nl, 1)
partial_google_trends_detrended_vectorized = np.ndarray.reshape(np.asarray(partial_google_trends_detrended)[np.triu_indices(m,1)], nl, 1)
mi_google_trends_detrended_vectorized = np.ndarray.reshape(np.asarray(mi_google_trends_detrended)[np.triu_indices(m,1)], nl, 1)
inv_ed_google_trends_detrended_vectorized = np.ndarray.reshape(np.asarray(inv_ed_google_trends_detrended)[np.triu_indices(m,1)], nl, 1)
inv_md_google_trends_detrended_vectorized = np.ndarray.reshape(np.asarray(inv_md_google_trends_detrended)[np.triu_indices(m,1)], nl, 1)

results_allmethods = np.concatenate((pearson_google_trends_vectorized, partial_partial_google_trends_vectorized, partial_google_trends_vectorized, \
                                     mi_google_trends_vectorized, inv_ed_google_trends_vectorized, inv_md_google_trends_vectorized, \
                                     pearson_google_trends_detrended_vectorized, partial_partial_google_trends_detrended_vectorized, partial_google_trends_detrended_vectorized, \
                                     mi_google_trends_detrended_vectorized, inv_ed_google_trends_detrended_vectorized, inv_md_google_trends_detrended_vectorized),axis=1)

corr_results_allmethods, corr_results_allmethods_p = sci.stats.spearmanr(results_allmethods)
labels_allmethods = ['Pearson correlation','Partial partial correlation', 'Partial partial correlation', \
          'Mutual information', 'Inverse Euclidean distance', 'Inverse Manhattan distance', \
          'Pearson correlation, detrended data','Partial partial correlation, detrended data', 'Partial partial correlation, detrended data', \
          'Mutual information, detrended data', 'Inverse Euclidean distance, detrended data', 'Inverse Manhattan distance, detrended data']


corr_results_allmethods = pd.DataFrame(corr_results_allmethods, index=labels_allmethods, columns=labels_allmethods)
corr_results_allmethods_p = pd.DataFrame(corr_results_allmethods_p, index=labels_allmethods, columns=labels_allmethods)

fig = plt.figure(figsize=(13,10))
ax0 = plt.subplot2grid((1,2),(0,0), colspan=1, rowspan=1)
ax1 = plt.subplot2grid((1,2),(0,1), colspan=1, rowspan=1)
sns.heatmap(corr_results_allmethods, vmin=-1.0, vmax=1.0, cmap='coolwarm', ax=ax0);
ax0.set_title('Spearman r between the methods', fontsize=16,y=1.70);
sns.heatmap(corr_results_allmethods_p, vmin=0, vmax=0.01, cmap='coolwarm', ax=ax1);
ax1.set_title('Associated p values', fontsize=16,  y=1.70);
for ax in fig.axes:
    plt.sca(ax)
    ax.axvline(x=6, color='k', linestyle='-')
    ax.axhline(y=6, color='k', linestyle='-')
    ax.xaxis.tick_top()
    plt.xticks(rotation=90)
for label in ax1.get_yticklabels():
    label.set_visible(False)
plt.tight_layout()

folder_output = output_google_trends
fname = folder_output + 'correlations_all_methods_google_trends'

dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

hp.savef(fig, fname, dpis, formats)

## Example on stock exchange data

In [ ]:
# choose the normalized version of the data:
path = foldername_data_stock_exchange + 'detrended-log.csv'
# read in only first 1,000 rows as the data is huge:
data_stock_exchange = pd.read_csv(path, header=0, index_col='date', nrows=1000)
labels = pd.Index.tolist(data_stock_exchange.columns)
data_stock_exchange = np.asarray(data_stock_exchange)

n = data_stock_exchange.shape[0]
m = data_stock_exchange.shape[1]

In [ ]:
# Pearson correlation:
pearson_stock_exchange, pearson_stock_exchange_p = (pd.DataFrame(fc.pearson_corr(data_stock_exchange)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.pearson_corr(data_stock_exchange)[1], 
                                          index=labels, columns=labels))
max_pearson_stock_exchange = np.max(fc.pearson_corr(data_stock_exchange)[0][np.triu_indices(m,1)])

# Partial partial correlation:
alpha = 0.5
partial_partial_stock_exchange, partial_partial_stock_exchange_p = (pd.DataFrame(fc.partial_partial_corr(data_stock_exchange,alpha)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.partial_partial_corr(data_stock_exchange,alpha)[1], 
                                          index=labels, columns=labels))
max_partial_partial_stock_exchange = np.max(fc.partial_partial_corr(data_stock_exchange,alpha)[0][np.triu_indices(m,1)])

# Partial correlation:
alpha = 1.0
partial_stock_exchange, partial_stock_exchange_p = (pd.DataFrame(fc.partial_partial_corr(data_stock_exchange,alpha)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.partial_partial_corr(data_stock_exchange,alpha)[1], 
                                          index=labels, columns=labels))
max_partial_stock_exchange = np.max(fc.partial_partial_corr(data_stock_exchange,alpha)[0][np.triu_indices(m,1)])

# Mutual Information:
# take number of bins as square root of the data length:
nbins = int(np.floor(np.sqrt(n)))
mi_stock_exchange = np.zeros((m,m))
for i in range(m):
    mi_stock_exchange[i, i] = 10
    for j in range(i+1, m):
        mi_stock_exchange[i,j] = fc.calc_MI(data_stock_exchange[:,i], data_stock_exchange[:,j], nbins)
        mi_stock_exchange[j,i] = mi_stock_exchange[i,j]

max_mi_stock_exchange = np.max(mi_stock_exchange[np.triu_indices(m,1)])
mi_stock_exchange = pd.DataFrame(mi_stock_exchange, index=labels, columns=labels)

# inverse of Euclidean distance:
inv_ed_stock_exchange = np.zeros((m,m))
for i in range(m):
    inv_ed_stock_exchange[i, i] = 1.0
    for j in range(i+1, m):
        inv_ed_stock_exchange[i,j] = 1.0/fc.euclidean_distance(data_stock_exchange[:,i], data_stock_exchange[:,j])
        inv_ed_stock_exchange[j,i] = inv_ed_stock_exchange[i,j]
        
max_inv_ed_stock_exchange = np.max(inv_ed_stock_exchange[np.triu_indices(m,1)])
inv_ed_stock_exchange = pd.DataFrame(inv_ed_stock_exchange, index=labels, columns=labels)

# inverse of Manhattan distance:
inv_md_stock_exchange = np.zeros((m,m))
for i in range(m):
    inv_md_stock_exchange[i, i] = 1.0
    for j in range(i+1, m):
        inv_md_stock_exchange[i,j] = 1.0/fc.manhattan_distance(data_stock_exchange[:,i], data_stock_exchange[:,j])
        inv_md_stock_exchange[j,i] = inv_md_stock_exchange[i,j]
        
max_inv_md_stock_exchange = np.max(inv_md_stock_exchange[np.triu_indices(m,1)])
inv_md_stock_exchange = pd.DataFrame(inv_md_stock_exchange, index=labels, columns=labels)

corr_stock_exchange = plot_corr(pearson_stock_exchange, pearson_stock_exchange_p, partial_partial_stock_exchange, partial_partial_stock_exchange_p, partial_stock_exchange, partial_stock_exchange_p)

all_stock_exchange = plot_all(pearson_stock_exchange, partial_partial_stock_exchange, partial_stock_exchange, mi_stock_exchange, inv_ed_stock_exchange, inv_md_stock_exchange, max_pearson_stock_exchange, max_partial_partial_stock_exchange, max_partial_stock_exchange, max_mi_stock_exchange, max_inv_ed_stock_exchange, max_inv_md_stock_exchange)

folder_output = output_stock_exchange
fname = folder_output + 'all_methods_stock_exchange'

dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

hp.savef(all_stock_exchange, fname, dpis, formats)

# plot and save all the clustermaps:
folder_output = output_stock_exchange
dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

# Pearson correlation:
fig1 = plt.figure(figsize=(12,11))
s1 = sns.clustermap(pearson_stock_exchange, vmin=-abs(max_pearson_stock_exchange), vmax=abs(max_pearson_stock_exchange), cmap='coolwarm');
plt.setp(s1.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s1.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Pearson correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_pearson_stock_exchange' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Partial partial correlation:
fig2 = plt.figure(figsize=(12,11))
s2 = sns.clustermap(partial_partial_stock_exchange, vmin=-abs(max_partial_partial_stock_exchange), vmax=abs(max_partial_partial_stock_exchange), cmap='coolwarm');
plt.setp(s2.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s2.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Partial partial correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_partial_partial_stock_exchange' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Partial correlation: 
fig3 = plt.figure(figsize=(12,11))
s3 = sns.clustermap(partial_stock_exchange, vmin=-abs(max_partial_stock_exchange), vmax=abs(max_partial_stock_exchange), cmap='coolwarm');
plt.setp(s3.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s3.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Partial correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_partial_stock_exchange' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Mutual information: 
fig4 = plt.figure(figsize=(12,11))
s4 = sns.clustermap(mi_stock_exchange, vmin=-abs(max_mi_stock_exchange), vmax=abs(max_mi_stock_exchange), cmap='coolwarm');
plt.setp(s4.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s4.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Mutual information', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_mi_stock_exchange' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

# Euclidean distance: 
fig5 = plt.figure(figsize=(12,11))
s5 = sns.clustermap(inv_ed_stock_exchange, vmin=-abs(max_inv_ed_stock_exchange), vmax=abs(max_inv_ed_stock_exchange), cmap='coolwarm');
plt.setp(s5.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s5.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Inverse Euclidean distance', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_inv_ed_stock_exchange' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

# Manhattan distance: 
fig6 = plt.figure(figsize=(12,11))
s6 = sns.clustermap(inv_md_stock_exchange, vmin=-abs(max_inv_md_stock_exchange), vmax=abs(max_inv_md_stock_exchange), cmap='coolwarm');
plt.setp(s6.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s6.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Inverse Manhattan distance', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_inv_md_stock_exchange' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

# add detrending:
data_stock_exchange_detrended = fc.detrend(data_stock_exchange)

# Pearson correlation:
pearson_stock_exchange_detrended, pearson_stock_exchange_p_detrended = (pd.DataFrame(fc.pearson_corr(data_stock_exchange_detrended)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.pearson_corr(data_stock_exchange_detrended)[1], 
                                          index=labels, columns=labels))
max_pearson_stock_exchange_detrended = np.max(fc.pearson_corr(data_stock_exchange_detrended)[0][np.triu_indices(m,1)])

# Partial partial correlation:
alpha = 0.5
partial_partial_stock_exchange_detrended, partial_partial_stock_exchange_p_detrended = (pd.DataFrame(fc.partial_partial_corr(data_stock_exchange_detrended,alpha)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.partial_partial_corr(data_stock_exchange_detrended,alpha)[1], 
                                          index=labels, columns=labels))
max_partial_partial_stock_exchange_detrended = np.max(fc.partial_partial_corr(data_stock_exchange_detrended,alpha)[0][np.triu_indices(m,1)])

# Partial correlation:
alpha = 1.0
partial_stock_exchange_detrended, partial_stock_exchange_p_detrended = (pd.DataFrame(fc.partial_partial_corr(data_stock_exchange_detrended,alpha)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.partial_partial_corr(data_stock_exchange_detrended,alpha)[1], 
                                          index=labels, columns=labels))
max_partial_stock_exchange_detrended = np.max(fc.partial_partial_corr(data_stock_exchange_detrended,alpha)[0][np.triu_indices(m,1)])

# Mutual Information:
# take number of bins as square root of the data length:
nbins = int(np.floor(np.sqrt(n)))
mi_stock_exchange_detrended = np.zeros((m,m))
for i in range(m):
    mi_stock_exchange_detrended[i, i] = 10
    for j in range(i+1, m):
        mi_stock_exchange_detrended[i,j] = fc.calc_MI(data_stock_exchange_detrended[:,i], data_stock_exchange_detrended[:,j], nbins)
        mi_stock_exchange_detrended[j,i] = mi_stock_exchange_detrended[i,j]

max_mi_stock_exchange_detrended = np.max(mi_stock_exchange_detrended[np.triu_indices(m,1)])
mi_stock_exchange_detrended = pd.DataFrame(mi_stock_exchange_detrended, index=labels, columns=labels)

# inverse of Euclidean distance:
inv_ed_stock_exchange_detrended = np.zeros((m,m))
for i in range(m):
    inv_ed_stock_exchange_detrended[i, i] = 1.0
    for j in range(i+1, m):
        inv_ed_stock_exchange_detrended[i,j] = 1.0/fc.euclidean_distance(data_stock_exchange_detrended[:,i], data_stock_exchange_detrended[:,j])
        inv_ed_stock_exchange_detrended[j,i] = inv_ed_stock_exchange_detrended[i,j]
        
max_inv_ed_stock_exchange_detrended = np.max(inv_ed_stock_exchange_detrended[np.triu_indices(m,1)])
inv_ed_stock_exchange_detrended = pd.DataFrame(inv_ed_stock_exchange_detrended, index=labels, columns=labels)

# inverse of Manhattan distance:
inv_md_stock_exchange_detrended = np.zeros((m,m))
for i in range(m):
    inv_md_stock_exchange_detrended[i, i] = 1.0
    for j in range(i+1, m):
        inv_md_stock_exchange_detrended[i,j] = 1.0/fc.manhattan_distance(data_stock_exchange_detrended[:,i], data_stock_exchange_detrended[:,j])
        inv_md_stock_exchange_detrended[j,i] = inv_md_stock_exchange_detrended[i,j]
        
max_inv_md_stock_exchange_detrended = np.max(inv_md_stock_exchange_detrended[np.triu_indices(m,1)])
inv_md_stock_exchange_detrended = pd.DataFrame(inv_md_stock_exchange_detrended, index=labels, columns=labels)

corr_stock_exchange_detrended = plot_corr(pearson_stock_exchange_detrended, pearson_stock_exchange_p_detrended, partial_partial_stock_exchange_detrended, partial_partial_stock_exchange_p_detrended, partial_stock_exchange_detrended, partial_stock_exchange_p_detrended)

all_stock_exchange_detrended = plot_all(pearson_stock_exchange_detrended, partial_partial_stock_exchange_detrended, partial_stock_exchange_detrended, mi_stock_exchange_detrended, inv_ed_stock_exchange_detrended, inv_md_stock_exchange_detrended, max_pearson_stock_exchange_detrended, max_partial_partial_stock_exchange_detrended, max_partial_stock_exchange_detrended, max_mi_stock_exchange_detrended, max_inv_ed_stock_exchange_detrended, max_inv_md_stock_exchange_detrended)

folder_output = output_stock_exchange
fname = folder_output + 'all_methods_stock_exchange_detrended'

dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

hp.savef(all_stock_exchange_detrended, fname, dpis, formats)

# plot and save all the clustermaps:
folder_output = output_stock_exchange
dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

# Pearson correlation:
fig1 = plt.figure(figsize=(12,11))
s1 = sns.clustermap(pearson_stock_exchange_detrended, vmin=-abs(max_pearson_stock_exchange_detrended), vmax=abs(max_pearson_stock_exchange_detrended), cmap='coolwarm');
plt.setp(s1.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s1.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Pearson correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_pearson_stock_exchange_detrended' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Partial partial correlation:
fig2 = plt.figure(figsize=(12,11))
s2 = sns.clustermap(partial_partial_stock_exchange_detrended, vmin=-abs(max_partial_partial_stock_exchange_detrended), vmax=abs(max_partial_partial_stock_exchange_detrended), cmap='coolwarm');
plt.setp(s2.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s2.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Partial partial correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_partial_partial_stock_exchange_detrended' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Partial correlation: 
fig3 = plt.figure(figsize=(12,11))
s3 = sns.clustermap(partial_stock_exchange_detrended, vmin=-abs(max_partial_stock_exchange_detrended), vmax=abs(max_partial_stock_exchange_detrended), cmap='coolwarm');
plt.setp(s3.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s3.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Partial correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_partial_stock_exchange_detrended' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Mutual information: 
fig4 = plt.figure(figsize=(12,11))
s4 = sns.clustermap(mi_stock_exchange_detrended, vmin=-abs(max_mi_stock_exchange_detrended), vmax=abs(max_mi_stock_exchange_detrended), cmap='coolwarm');
plt.setp(s4.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s4.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Mutual information', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_mi_stock_exchange_detrended' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

# Euclidean distance: 
fig5 = plt.figure(figsize=(12,11))
s5 = sns.clustermap(inv_ed_stock_exchange_detrended, vmin=-abs(max_inv_ed_stock_exchange_detrended), vmax=abs(max_inv_ed_stock_exchange_detrended), cmap='coolwarm');
plt.setp(s5.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s5.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Inverse Euclidean distance', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_inv_ed_stock_exchange_detrended' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

# Manhattan distance: 
fig6 = plt.figure(figsize=(12,11))
s6 = sns.clustermap(inv_md_stock_exchange_detrended, vmin=-abs(max_inv_md_stock_exchange_detrended), vmax=abs(max_inv_md_stock_exchange_detrended), cmap='coolwarm');
plt.setp(s6.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s6.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Inverse Manhattan distance', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_inv_md_stock_exchange_detrended' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

# plot correlation between the 12 methods:
nl = len(np.triu_indices(m,1)[0])

pearson_stock_exchange_vectorized = np.ndarray.reshape(np.asarray(pearson_stock_exchange)[np.triu_indices(m,1)], nl, 1)
partial_partial_stock_exchange_vectorized = np.ndarray.reshape(np.asarray(partial_partial_stock_exchange)[np.triu_indices(m,1)], nl, 1)
partial_stock_exchange_vectorized = np.ndarray.reshape(np.asarray(partial_stock_exchange)[np.triu_indices(m,1)], nl, 1)
mi_stock_exchange_vectorized = np.ndarray.reshape(np.asarray(mi_stock_exchange)[np.triu_indices(m,1)], nl, 1)
inv_ed_stock_exchange_vectorized = np.ndarray.reshape(np.asarray(inv_ed_stock_exchange)[np.triu_indices(m,1)], nl, 1)
inv_md_stock_exchange_vectorized = np.ndarray.reshape(np.asarray(inv_md_stock_exchange)[np.triu_indices(m,1)], nl, 1)

pearson_stock_exchange_detrended_vectorized = np.ndarray.reshape(np.asarray(pearson_stock_exchange_detrended)[np.triu_indices(m,1)], nl, 1)
partial_partial_stock_exchange_detrended_vectorized = np.ndarray.reshape(np.asarray(partial_partial_stock_exchange_detrended)[np.triu_indices(m,1)], nl, 1)
partial_stock_exchange_detrended_vectorized = np.ndarray.reshape(np.asarray(partial_stock_exchange_detrended)[np.triu_indices(m,1)], nl, 1)
mi_stock_exchange_detrended_vectorized = np.ndarray.reshape(np.asarray(mi_stock_exchange_detrended)[np.triu_indices(m,1)], nl, 1)
inv_ed_stock_exchange_detrended_vectorized = np.ndarray.reshape(np.asarray(inv_ed_stock_exchange_detrended)[np.triu_indices(m,1)], nl, 1)
inv_md_stock_exchange_detrended_vectorized = np.ndarray.reshape(np.asarray(inv_md_stock_exchange_detrended)[np.triu_indices(m,1)], nl, 1)

results_allmethods = np.concatenate((pearson_stock_exchange_vectorized, partial_partial_stock_exchange_vectorized, partial_stock_exchange_vectorized, \
                                     mi_stock_exchange_vectorized, inv_ed_stock_exchange_vectorized, inv_md_stock_exchange_vectorized, \
                                     pearson_stock_exchange_detrended_vectorized, partial_partial_stock_exchange_detrended_vectorized, partial_stock_exchange_detrended_vectorized, \
                                     mi_stock_exchange_detrended_vectorized, inv_ed_stock_exchange_detrended_vectorized, inv_md_stock_exchange_detrended_vectorized),axis=1)

corr_results_allmethods, corr_results_allmethods_p = sci.stats.spearmanr(results_allmethods)
labels_allmethods = ['Pearson correlation','Partial partial correlation', 'Partial partial correlation', \
          'Mutual information', 'Inverse Euclidean distance', 'Inverse Manhattan distance', \
          'Pearson correlation, detrended data','Partial partial correlation, detrended data', 'Partial partial correlation, detrended data', \
          'Mutual information, detrended data', 'Inverse Euclidean distance, detrended data', 'Inverse Manhattan distance, detrended data']


corr_results_allmethods = pd.DataFrame(corr_results_allmethods, index=labels_allmethods, columns=labels_allmethods)
corr_results_allmethods_p = pd.DataFrame(corr_results_allmethods_p, index=labels_allmethods, columns=labels_allmethods)

fig = plt.figure(figsize=(13,10))
ax0 = plt.subplot2grid((1,2),(0,0), colspan=1, rowspan=1)
ax1 = plt.subplot2grid((1,2),(0,1), colspan=1, rowspan=1)
sns.heatmap(corr_results_allmethods, vmin=-1.0, vmax=1.0, cmap='coolwarm', ax=ax0);
ax0.set_title('Spearman r between the methods', fontsize=16,y=1.70);
sns.heatmap(corr_results_allmethods_p, vmin=0, vmax=0.01, cmap='coolwarm', ax=ax1);
ax1.set_title('Associated p values', fontsize=16,  y=1.70);
for ax in fig.axes:
    plt.sca(ax)
    ax.axvline(x=6, color='k', linestyle='-')
    ax.axhline(y=6, color='k', linestyle='-')
    ax.xaxis.tick_top()
    plt.xticks(rotation=90)
for label in ax1.get_yticklabels():
    label.set_visible(False)
plt.tight_layout()

folder_output = output_stock_exchange
fname = folder_output + 'correlations_all_methods_stock_exchange'

dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

hp.savef(fig, fname, dpis, formats)

## Example on weather data

In [ ]:
# choose the normalized version of the data:
path = foldername_data_weather + 'data_by_airport/5_LosAngeles.csv'
data_weather0 = pd.read_csv(path, sep=';', usecols=[2,3,5,8,11,13], index_col='Date').dropna()
labels = pd.Index.tolist(data_weather0.columns)
data_weather0 = np.asarray(data_weather0)

n = data_weather0.shape[0]
m = data_weather0.shape[1]

In [ ]:
# normalize the data with boxcox:
# add little number to the data (so that all the values are positive):
data_weather = data_weather0 + 1e-6*np.ones(data_weather0.shape)

for ind in range(m):
    boxcoxed = sci.stats.boxcox(np.ndarray.reshape(data_weather[:,ind], n, 1))
    data_weather[:,ind] = boxcoxed[0].reshape((n,))

In [ ]:
# Pearson correlation:
pearson_weather, pearson_weather_p = (pd.DataFrame(fc.pearson_corr(data_weather)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.pearson_corr(data_weather)[1], 
                                          index=labels, columns=labels))
max_pearson_weather = np.max(fc.pearson_corr(data_weather)[0][np.triu_indices(m,1)])

# Partial partial correlation:
alpha = 0.5
partial_partial_weather, partial_partial_weather_p = (pd.DataFrame(fc.partial_partial_corr(data_weather,alpha)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.partial_partial_corr(data_weather,alpha)[1], 
                                          index=labels, columns=labels))
max_partial_partial_weather = np.max(fc.partial_partial_corr(data_weather,alpha)[0][np.triu_indices(m,1)])

# Partial correlation:
alpha = 1.0
partial_weather, partial_weather_p = (pd.DataFrame(fc.partial_partial_corr(data_weather,alpha)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.partial_partial_corr(data_weather,alpha)[1], 
                                          index=labels, columns=labels))
max_partial_weather = np.max(fc.partial_partial_corr(data_weather,alpha)[0][np.triu_indices(m,1)])

# Mutual Information:
# take number of bins as square root of the data length:
nbins = int(np.floor(np.sqrt(n)))
mi_weather = np.zeros((m,m))
for i in range(m):
    mi_weather[i, i] = 10
    for j in range(i+1, m):
        mi_weather[i,j] = fc.calc_MI(data_weather[:,i], data_weather[:,j], nbins)
        mi_weather[j,i] = mi_weather[i,j]

max_mi_weather = np.max(mi_weather[np.triu_indices(m,1)])
mi_weather = pd.DataFrame(mi_weather, index=labels, columns=labels)

# inverse of Euclidean distance:
inv_ed_weather = np.zeros((m,m))
for i in range(m):
    inv_ed_weather[i, i] = 1.0
    for j in range(i+1, m):
        inv_ed_weather[i,j] = 1.0/fc.euclidean_distance(data_weather[:,i], data_weather[:,j])
        inv_ed_weather[j,i] = inv_ed_weather[i,j]
        
max_inv_ed_weather = np.max(inv_ed_weather[np.triu_indices(m,1)])
inv_ed_weather = pd.DataFrame(inv_ed_weather, index=labels, columns=labels)

# inverse of Manhattan distance:
inv_md_weather = np.zeros((m,m))
for i in range(m):
    inv_md_weather[i, i] = 1.0
    for j in range(i+1, m):
        inv_md_weather[i,j] = 1.0/fc.manhattan_distance(data_weather[:,i], data_weather[:,j])
        inv_md_weather[j,i] = inv_md_weather[i,j]
        
max_inv_md_weather = np.max(inv_md_weather[np.triu_indices(m,1)])
inv_md_weather = pd.DataFrame(inv_md_weather, index=labels, columns=labels)

corr_weather = plot_corr(pearson_weather, pearson_weather_p, partial_partial_weather, partial_partial_weather_p, partial_weather, partial_weather_p)

all_weather = plot_all(pearson_weather, partial_partial_weather, partial_weather, mi_weather, inv_ed_weather, inv_md_weather, max_pearson_weather, max_partial_partial_weather, max_partial_weather, max_mi_weather, max_inv_ed_weather, max_inv_md_weather)

folder_output = output_weather
fname = folder_output + 'all_methods_weather'

dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

hp.savef(all_weather, fname, dpis, formats)

# plot and save all the clustermaps:
folder_output = output_weather
dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

# Pearson correlation:
fig1 = plt.figure(figsize=(12,11))
s1 = sns.clustermap(pearson_weather, vmin=-abs(max_pearson_weather), vmax=abs(max_pearson_weather), cmap='coolwarm');
plt.setp(s1.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s1.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Pearson correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_pearson_weather' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Partial partial correlation:
fig2 = plt.figure(figsize=(12,11))
s2 = sns.clustermap(partial_partial_weather, vmin=-abs(max_partial_partial_weather), vmax=abs(max_partial_partial_weather), cmap='coolwarm');
plt.setp(s2.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s2.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Partial partial correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_partial_partial_weather' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Partial correlation: 
fig3 = plt.figure(figsize=(12,11))
s3 = sns.clustermap(partial_weather, vmin=-abs(max_partial_weather), vmax=abs(max_partial_weather), cmap='coolwarm');
plt.setp(s3.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s3.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Partial correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_partial_weather' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Mutual information: 
fig4 = plt.figure(figsize=(12,11))
s4 = sns.clustermap(mi_weather, vmin=-abs(max_mi_weather), vmax=abs(max_mi_weather), cmap='coolwarm');
plt.setp(s4.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s4.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Mutual information', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_mi_weather' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

# Euclidean distance: 
fig5 = plt.figure(figsize=(12,11))
s5 = sns.clustermap(inv_ed_weather, vmin=-abs(max_inv_ed_weather), vmax=abs(max_inv_ed_weather), cmap='coolwarm');
plt.setp(s5.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s5.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Inverse Euclidean distance', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_inv_ed_weather' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

# Manhattan distance: 
fig6 = plt.figure(figsize=(12,11))
s6 = sns.clustermap(inv_md_weather, vmin=-abs(max_inv_md_weather), vmax=abs(max_inv_md_weather), cmap='coolwarm');
plt.setp(s6.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s6.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Inverse Manhattan distance', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_inv_md_weather' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

# add detrending:
data_weather_detrended = fc.detrend(data_weather)

# Pearson correlation:
pearson_weather_detrended, pearson_weather_p_detrended = (pd.DataFrame(fc.pearson_corr(data_weather_detrended)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.pearson_corr(data_weather_detrended)[1], 
                                          index=labels, columns=labels))
max_pearson_weather_detrended = np.max(fc.pearson_corr(data_weather_detrended)[0][np.triu_indices(m,1)])

# Partial partial correlation:
alpha = 0.5
partial_partial_weather_detrended, partial_partial_weather_p_detrended = (pd.DataFrame(fc.partial_partial_corr(data_weather_detrended,alpha)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.partial_partial_corr(data_weather_detrended,alpha)[1], 
                                          index=labels, columns=labels))
max_partial_partial_weather_detrended = np.max(fc.partial_partial_corr(data_weather_detrended,alpha)[0][np.triu_indices(m,1)])

# Partial correlation:
alpha = 1.0
partial_weather_detrended, partial_weather_p_detrended = (pd.DataFrame(fc.partial_partial_corr(data_weather_detrended,alpha)[0], 
                                          index=labels, columns=labels),
                                pd.DataFrame(fc.partial_partial_corr(data_weather_detrended,alpha)[1], 
                                          index=labels, columns=labels))
max_partial_weather_detrended = np.max(fc.partial_partial_corr(data_weather_detrended,alpha)[0][np.triu_indices(m,1)])

# Mutual Information:
# take number of bins as square root of the data length:
nbins = int(np.floor(np.sqrt(n)))
mi_weather_detrended = np.zeros((m,m))
for i in range(m):
    mi_weather_detrended[i, i] = 10
    for j in range(i+1, m):
        mi_weather_detrended[i,j] = fc.calc_MI(data_weather_detrended[:,i], data_weather_detrended[:,j], nbins)
        mi_weather_detrended[j,i] = mi_weather_detrended[i,j]

max_mi_weather_detrended = np.max(mi_weather_detrended[np.triu_indices(m,1)])
mi_weather_detrended = pd.DataFrame(mi_weather_detrended, index=labels, columns=labels)

# inverse of Euclidean distance:
inv_ed_weather_detrended = np.zeros((m,m))
for i in range(m):
    inv_ed_weather_detrended[i, i] = 1.0
    for j in range(i+1, m):
        inv_ed_weather_detrended[i,j] = 1.0/fc.euclidean_distance(data_weather_detrended[:,i], data_weather_detrended[:,j])
        inv_ed_weather_detrended[j,i] = inv_ed_weather_detrended[i,j]
        
max_inv_ed_weather_detrended = np.max(inv_ed_weather_detrended[np.triu_indices(m,1)])
inv_ed_weather_detrended = pd.DataFrame(inv_ed_weather_detrended, index=labels, columns=labels)

# inverse of Manhattan distance:
inv_md_weather_detrended = np.zeros((m,m))
for i in range(m):
    inv_md_weather_detrended[i, i] = 1.0
    for j in range(i+1, m):
        inv_md_weather_detrended[i,j] = 1.0/fc.manhattan_distance(data_weather_detrended[:,i], data_weather_detrended[:,j])
        inv_md_weather_detrended[j,i] = inv_md_weather_detrended[i,j]
        
max_inv_md_weather_detrended = np.max(inv_md_weather_detrended[np.triu_indices(m,1)])
inv_md_weather_detrended = pd.DataFrame(inv_md_weather_detrended, index=labels, columns=labels)

corr_weather_detrended = plot_corr(pearson_weather_detrended, pearson_weather_p_detrended, partial_partial_weather_detrended, partial_partial_weather_p_detrended, partial_weather_detrended, partial_weather_p_detrended)

all_weather_detrended = plot_all(pearson_weather_detrended, partial_partial_weather_detrended, partial_weather_detrended, mi_weather_detrended, inv_ed_weather_detrended, inv_md_weather_detrended, max_pearson_weather_detrended, max_partial_partial_weather_detrended, max_partial_weather_detrended, max_mi_weather_detrended, max_inv_ed_weather_detrended, max_inv_md_weather_detrended)

folder_output = output_weather
fname = folder_output + 'all_methods_weather_detrended'

dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

hp.savef(all_weather_detrended, fname, dpis, formats)

# plot and save all the clustermaps:
folder_output = output_weather
dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

# Pearson correlation:
fig1 = plt.figure(figsize=(12,11))
s1 = sns.clustermap(pearson_weather_detrended, vmin=-abs(max_pearson_weather_detrended), vmax=abs(max_pearson_weather_detrended), cmap='coolwarm');
plt.setp(s1.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s1.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Pearson correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_pearson_weather_detrended' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Partial partial correlation:
fig2 = plt.figure(figsize=(12,11))
s2 = sns.clustermap(partial_partial_weather_detrended, vmin=-abs(max_partial_partial_weather_detrended), vmax=abs(max_partial_partial_weather_detrended), cmap='coolwarm');
plt.setp(s2.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s2.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Partial partial correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_partial_partial_weather_detrended' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Partial correlation: 
fig3 = plt.figure(figsize=(12,11))
s3 = sns.clustermap(partial_weather_detrended, vmin=-abs(max_partial_weather_detrended), vmax=abs(max_partial_weather_detrended), cmap='coolwarm');
plt.setp(s3.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s3.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Partial correlation', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_partial_weather_detrended' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")
        
# Mutual information: 
fig4 = plt.figure(figsize=(12,11))
s4 = sns.clustermap(mi_weather_detrended, vmin=-abs(max_mi_weather_detrended), vmax=abs(max_mi_weather_detrended), cmap='coolwarm');
plt.setp(s4.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s4.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Mutual information', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_mi_weather_detrended' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

# Euclidean distance: 
fig5 = plt.figure(figsize=(12,11))
s5 = sns.clustermap(inv_ed_weather_detrended, vmin=-abs(max_inv_ed_weather_detrended), vmax=abs(max_inv_ed_weather_detrended), cmap='coolwarm');
plt.setp(s5.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s5.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Inverse Euclidean distance', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_inv_ed_weather_detrended' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

# Manhattan distance: 
fig6 = plt.figure(figsize=(12,11))
s6 = sns.clustermap(inv_md_weather_detrended, vmin=-abs(max_inv_md_weather_detrended), vmax=abs(max_inv_md_weather_detrended), cmap='coolwarm');
plt.setp(s6.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(s6.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.title('Inverse Manhattan distance', fontsize=16, y=1.10);
for i in range(len(dpis)):
    for j in range(len(formats)):
        fname = folder_output + 'clustermap_inv_md_weather_detrended' + \
        str(dpis[i]) + '.' + formats[j]
        plt.savefig(fname, dpi=dpis[i], format=formats[j], bbox_inches="tight")

# plot correlation between the 12 methods:
nl = len(np.triu_indices(m,1)[0])

pearson_weather_vectorized = np.ndarray.reshape(np.asarray(pearson_weather)[np.triu_indices(m,1)], nl, 1)
partial_partial_weather_vectorized = np.ndarray.reshape(np.asarray(partial_partial_weather)[np.triu_indices(m,1)], nl, 1)
partial_weather_vectorized = np.ndarray.reshape(np.asarray(partial_weather)[np.triu_indices(m,1)], nl, 1)
mi_weather_vectorized = np.ndarray.reshape(np.asarray(mi_weather)[np.triu_indices(m,1)], nl, 1)
inv_ed_weather_vectorized = np.ndarray.reshape(np.asarray(inv_ed_weather)[np.triu_indices(m,1)], nl, 1)
inv_md_weather_vectorized = np.ndarray.reshape(np.asarray(inv_md_weather)[np.triu_indices(m,1)], nl, 1)

pearson_weather_detrended_vectorized = np.ndarray.reshape(np.asarray(pearson_weather_detrended)[np.triu_indices(m,1)], nl, 1)
partial_partial_weather_detrended_vectorized = np.ndarray.reshape(np.asarray(partial_partial_weather_detrended)[np.triu_indices(m,1)], nl, 1)
partial_weather_detrended_vectorized = np.ndarray.reshape(np.asarray(partial_weather_detrended)[np.triu_indices(m,1)], nl, 1)
mi_weather_detrended_vectorized = np.ndarray.reshape(np.asarray(mi_weather_detrended)[np.triu_indices(m,1)], nl, 1)
inv_ed_weather_detrended_vectorized = np.ndarray.reshape(np.asarray(inv_ed_weather_detrended)[np.triu_indices(m,1)], nl, 1)
inv_md_weather_detrended_vectorized = np.ndarray.reshape(np.asarray(inv_md_weather_detrended)[np.triu_indices(m,1)], nl, 1)

results_allmethods = np.concatenate((pearson_weather_vectorized, partial_partial_weather_vectorized, partial_weather_vectorized, \
                                     mi_weather_vectorized, inv_ed_weather_vectorized, inv_md_weather_vectorized, \
                                     pearson_weather_detrended_vectorized, partial_partial_weather_detrended_vectorized, partial_weather_detrended_vectorized, \
                                     mi_weather_detrended_vectorized, inv_ed_weather_detrended_vectorized, inv_md_weather_detrended_vectorized),axis=1)

corr_results_allmethods, corr_results_allmethods_p = sci.stats.spearmanr(results_allmethods)
labels_allmethods = ['Pearson correlation','Partial partial correlation', 'Partial partial correlation', \
          'Mutual information', 'Inverse Euclidean distance', 'Inverse Manhattan distance', \
          'Pearson correlation, detrended data','Partial partial correlation, detrended data', 'Partial partial correlation, detrended data', \
          'Mutual information, detrended data', 'Inverse Euclidean distance, detrended data', 'Inverse Manhattan distance, detrended data']


corr_results_allmethods = pd.DataFrame(corr_results_allmethods, index=labels_allmethods, columns=labels_allmethods)
corr_results_allmethods_p = pd.DataFrame(corr_results_allmethods_p, index=labels_allmethods, columns=labels_allmethods)

fig = plt.figure(figsize=(13,10))
ax0 = plt.subplot2grid((1,2),(0,0), colspan=1, rowspan=1)
ax1 = plt.subplot2grid((1,2),(0,1), colspan=1, rowspan=1)
sns.heatmap(corr_results_allmethods, vmin=-1.0, vmax=1.0, cmap='coolwarm', ax=ax0);
ax0.set_title('Spearman r between the methods', fontsize=16,y=1.70);
sns.heatmap(corr_results_allmethods_p, vmin=0, vmax=0.01, cmap='coolwarm', ax=ax1);
ax1.set_title('Associated p values', fontsize=16,  y=1.70);
for ax in fig.axes:
    plt.sca(ax)
    ax.axvline(x=6, color='k', linestyle='-')
    ax.axhline(y=6, color='k', linestyle='-')
    ax.xaxis.tick_top()
    plt.xticks(rotation=90)
for label in ax1.get_yticklabels():
    label.set_visible(False)
plt.tight_layout()

folder_output = output_weather
fname = folder_output + 'correlations_all_methods_weather'

dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

hp.savef(fig, fname, dpis, formats)

In [ ]:
path = + 'data_by_airport/5_LosAngeles.csv'

In [ ]:
pearson_weather, pearson_weather_p = fc.pearson_corr(pd.read_csv(path, sep=';', usecols=[2,3,5,8,11,13], index_col='Date').dropna())

In [ ]:
partial_weather, partial_weather_p = fc.partial_corr(pd.read_csv(path, sep=';', usecols=[2,3,5,8,11,13], index_col='Date').dropna())

In [ ]:
corr_weather = plot_corr(pearson_weather, pearson_weather_p, partial_weather, partial_weather_p)

In [ ]:
folder_output = '../results/1_Pearson_vs_Partial/'
fname = folder_output + 'corr_weather'

dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

hp.savef(corr_weather, fname, dpis, formats)

### Berkson's Paradox

In [ ]:
x1 = np.random.randint(0,1000,100)
x3 = np.random.randint(0,1000,100)
x2 = 0.5*x1 + 0.5*x3
labels = ['1','2','3']
x = np.arange(3)+0.5

In [ ]:
xxx = np.column_stack([x1,x2,x3])

In [ ]:
fig = plt.figure(figsize=(7,6))
sns.heatmap(fc.pearson_corr(xxx)[0], vmin=-1.0, vmax=1.0, cmap='coolwarm');
ax = plt.axes()
ax.set_title('Pearson correlation')
plt.xticks(x, labels)
plt.yticks(x, labels)
plt.tight_layout()
plt.show()

folder_output = output_Berkson
fname = folder_output + 'Berkson_1'

dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

hp.savef(fig, fname, dpis, formats)

In [ ]:
alpha = 0.5
fig = plt.figure(figsize=(7,6))
sns.heatmap(fc.partial_partial_corr(xxx,alpha)[0], vmin=-0.5, vmax=1, cmap='coolwarm');
ax = plt.axes()
ax.set_title('Partial partial correlation, alpha = 0.5')
plt.xticks(x, labels)
plt.yticks(x, labels)
plt.tight_layout()
plt.show()

folder_output = output_Berkson
fname = folder_output + 'Berkson_2'

dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

hp.savef(fig, fname, dpis, formats)

In [ ]:
alpha = 1.0
fig = plt.figure(figsize=(7,6))
sns.heatmap(fc.partial_partial_corr(xxx,alpha)[0], vmin=-0.5, vmax=1, cmap='coolwarm');
ax = plt.axes()
ax.set_title('Partial correlation')
plt.xticks(x, labels)
plt.yticks(x, labels)
plt.tight_layout()
plt.show()

folder_output = output_Berkson
fname = folder_output + 'Berkson_3'

dpis = [300, 600]
formats = ['jpg', 'png', 'pdf']

hp.savef(fig, fname, dpis, formats)